In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import datetime
import numpy as np
# from tqdm import tqdm
import warnings
from glob import glob
import tensorflow as tf
# from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow import keras
from tensorflow.keras import layers
# from sklearn.model_selection import train_test_split
# from pandasql import sqldf
from sklearn.preprocessing import StandardScaler
import os
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
warnings.filterwarnings(action='ignore')
from matplotlib import font_manager, rc
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

# font_path = "C:\\Windows\\Fonts\\\x7f\x7f\x7f\x7fBOLD.TTF"
# font = font_manager.FontProperties(fname=font_path).get_name()
# rc('font', family=font)
from sklearn.manifold import TSNE
import re
# import talib as tb

#-------------------- 주피터 , 출력결과 넓이 늘리기 ---------------
from IPython.core.display import display, HTML
display(HTML("<style>.container{width:100% !important;}</style>"))
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('max_colwidth', None)

In [3]:
df_레이아웃 = pd.read_excel('/content/drive/MyDrive/nice공모전/data/넘블 챌린지 개방데이터 레이아웃.xlsx')
df_레이아웃.head()

,순서,테이블명,영문칼럼명,한글칼럼명,칼럼형식,디폴트값,NULL여부,비고
0,1.0,기업개요,BIZ_NO,사업자등록번호,VARCHAR2(10),'Z',N,NaN
1,2.0,기업개요,CMP_PFIX_NM,기업접두명,VARCHAR2(64),NaN,Y,NaN
2,3.0,기업개요,CMP_NM,기업명,VARCHAR2(128),NaN,Y,NaN
3,4.0,기업개요,CMP_SFIX_NM,기업접미명,VARCHAR2(64),NaN,Y,NaN
4,5.0,기업개요,CMP_ENM,기업영문명,VARCHAR2(256),NaN,Y,NaN


In [4]:
df_코드집 = pd.read_excel('/content/drive/MyDrive/nice공모전/data/넘블 챌린지 개방데이터 코드집.xlsx', sheet_name = '코드집')
df_코드집.head()


,GRP_CD,CD,END_DATE,STRT_DATE,CD_NM
0,BZ_TYP,A,99991231,20191201,신금(신용금고)
1,BZ_TYP,B,99991231,20191201,은행
2,BZ_TYP,C,99991231,20191201,카드
3,BZ_TYP,F,99991231,20191201,생보(생명보험)
4,BZ_TYP,I,99991231,20191201,손보(손해보험)


In [5]:
df_co_재무계정 = pd.read_excel('/content/drive/MyDrive/nice공모전/data/넘블 챌린지 개방데이터 코드집.xlsx', sheet_name = '재무계정')
df_co_재무계정.head()

,계정코드,순번,계정명,영문
0,M11000,1,유동자산,Current Assets
1,M11001,2,*국고보조금,*Governments Grants
2,M11002,3,현금,Cash
3,M11003,4,외화,Foreign Currencies
4,M11004,5,예금,Bank Deposits


In [374]:
# df_재무데이터 = pd.read_excel('/content/drive/MyDrive/넘블휴폐업예측/넘블_재무데이터.xlsx')
# df_재무데이터

In [6]:
df_액티브중소법인재무_기업개요외감 = pd.read_csv('/content/drive/MyDrive/nice공모전/data/액티브 중소법인 재무보유00.csv')
                                   #,  converters= {'ESTB_DATE': pd.to_datetime}  )  #parse_dates=['ESTB_DATE'])
# df_액티브중소법인재무_기업개요외감.to_csv("./df_액티브중소법인재무_기업개요외감.csv", index=False)

In [7]:
df_액티브중소법인재무_기업개요 = pd.read_excel('/content/drive/MyDrive/nice공모전/data/액티브 중소법인 재무보유.xlsx', sheet_name = '기업개요')
df_액티브중소법인재무_기업개요.head()

,BIZ_NO,CMP_PFIX_NM,CMP_NM,CMP_NM1,CMP_SFIX_NM,CMP_ENM,BZ_TYP,CMP_TYP,CMP_SCL,PBCO_GB,PSN_CORP_GB,HDOF_BR_GB,FR_IVST_CORP_YN,VENT_YN,LIST_CD,IND_CD_ORDR,IND_CD1,IND_CD2,IND_CD3,PB_ORG_TYP,MDSCO_PRTC_YN,TAXN_TYP,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,MN_BIZ_CONT,NATN_NM,HOMEPAGE_URL,CEO_NM,EMP_CNT
0,2218116923,NaN,NaN,신우종합건설,(주),"Shinu General Construction Co., Ltd.",M,4,2,2,1,1,N,N,NaN,10.0,41200.0,NaN,NaN,0,N,3.0,NaN,20000807.0,1.0,NaN,토목공사업,NaN,NaN,김주대외 1명,12.0
1,1138111739,NaN,한일가스산업(주),한일가스산업,(주),"HANIL GAS IND. CO.,LTD.",M,3,2,2,1,1,N,N,NaN,10.0,47712.0,NaN,NaN,0,N,3.0,NaN,19660419.0,1.0,NaN,LP Gas 충전 및 판매업,NaN,www.hanilgas.com,홍순철외 1명,43.0
2,1108146490,(주),NaN,랩솔루션,NaN,Lab Solution,M,4,2,2,1,1,N,N,NaN,10.0,27210.0,27213.0,NaN,0,N,3.0,NaN,20000612.0,1.0,NaN,기체분석기 등,NaN,www.lab-solution.com,김봉철,3.0
3,3128134737,(주),NaN,고려인삼제품공사,NaN,"KOREAN GINSENG PRODUCT CO., LTD",M,4,2,2,1,1,N,N,NaN,10.0,10795.0,NaN,NaN,0,N,3.0,NaN,19920301.0,1.0,NaN,"다류, 차, 인삼제품, 음료, 코코아가공품, 커피",NaN,www.kitea.co.kr,박경출,24.0
4,2078132193,(주),(주)엠피아이,엠피아이,NaN,"EMPIALEE CO.,LTD.",M,3,2,2,1,1,N,N,NaN,10.0,15190.0,NaN,NaN,0,N,3.0,NaN,19910806.0,1.0,NaN,가죽 및 대용가죽제품 제조업,NaN,www.mastercnd.co.kr,이스테판상수외 1명,35.0


In [8]:
df_액티브중소법인재무_휴폐업이력 = pd.read_excel('/content/drive/MyDrive/nice공모전/data/액티브 중소법인 재무보유.xlsx', sheet_name = '휴폐업이력')
df_액티브중소법인재무_휴폐업이력.head()

,BIZ_NO,END_DATE,STRT_DATE,CLSBZ_GB,STAT_OCR_DATE
0,5088115592,99991231,20150209,5,NaN
1,5088115592,20150208,20200731,2,NaN
2,1138127100,99991231,20121105,5,NaN
3,1138127100,20121104,20200731,2,NaN
4,1138127100,20121004,20200731,5,NaN


In [9]:
df_휴폐업중소법인재무_기업개요 = pd.read_excel('/content/drive/MyDrive/nice공모전/data/휴폐업 중소법인 재무보유.xlsx', sheet_name = '기업개요')
df_휴폐업중소법인재무_기업개요.head()

,BIZ_NO,CMP_PFIX_NM,CMP_NM,CMP_SFIX_NM,CMP_ENM,BZ_TYP,CMP_SCL,PBCO_GB,PSN_CORP_GB,HDOF_BR_GB,FR_IVST_CORP_YN,VENT_YN,LIST_CD,IND_CD_ORDR,IND_CD1,IND_CD2,IND_CD3,PB_ORG_TYP,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,MN_BIZ_CONT,NATN_NM,HOMEPAGE_URL,CEO_NM,EMP_CNT
0,1268145633,(주),용진테크,NaN,"Yongjin Tech Co., Ltd.",M,2,2,1,1,N,N,NaN,10.0,30391.0,NaN,NaN,0,N,NaN,20000529.0,1,NaN,자동차부품 제조판매업,NaN,www.yongjintech.com,안상동,40.0
1,1388101749,NaN,신구건설,(주),"SHINKU C0NSTRUCTION CO.,LTD.",M,2,2,1,1,N,N,NaN,10.0,46610.0,NaN,NaN,0,N,NaN,19850110.0,1,NaN,건축자재 판매업,NaN,www.shinku.co.kr,소재필,13.0
2,1068179240,(주),석정미디어,NaN,sung jung,M,2,2,1,1,N,N,NaN,10.0,46510.0,NaN,NaN,0,N,NaN,19990601.0,1,NaN,"토너, 잉크, 프린터 외 컴퓨터 관련 주변기기",NaN,NaN,이상호,1.0
3,2248114507,(주),새한실업,NaN,"SAEHAN INDUSTRY CO.,LTD.",M,2,2,1,1,N,N,NaN,10.0,25991.0,NaN,NaN,0,N,NaN,19980522.0,1,NaN,금속 캔 및 기타 포장용기 제조업,NaN,NaN,김정남외 1명,6.0
4,1168132785,(주),석영엔터프라이즈,NaN,"Seokyoung Enterprise Co.,Ltd.",M,2,2,1,1,N,N,NaN,10.0,24199.0,NaN,NaN,0,N,NaN,19930226.0,1,NaN,스마트트레이 외,NaN,www.systeel.co.kr,박성진,40.0


In [10]:
df_휴폐업중소법인재무_휴폐업이력 = pd.read_excel('/content/drive/MyDrive/nice공모전/data/휴폐업 중소법인 재무보유.xlsx', sheet_name = '휴폐업이력')
df_휴폐업중소법인재무_휴폐업이력.head()

,BIZ_NO,END_DATE,STRT_DATE,CLSBZ_GB,STAT_OCR_DATE
0,1268145633,99991231,20220518,3,20220518
1,1388101749,99991231,20210319,3,20210319
2,1068179240,99991231,20200731,3,20191213
3,2248114507,99991231,20210531,3,20210531
4,1168132785,99991231,20210630,3,20210630


In [11]:
df_휴폐업중소법인재무_휴폐업이력['END_DATE'].value_counts() # 잘나오는거 확인 

99991231    6739
20200607       6
20200507       4
20190527       3
20191209       3
20200817       2
20201123       2
20210824       2
20211026       2
20200410       2
20201112       2
20210704       2
20200217       1
20220601       1
20190901       1
20191121       1
20211214       1
20200408       1
20190626       1
20200309       1
20201209       1
20211230       1
20211013       1
20220203       1
20200621       1
20200608       1
20200512       1
20190506       1
20200310       1
20201222       1
20220530       1
20201103       1
20180507       1
20180627       1
20190729       1
20220126       1
20220802       1
20201007       1
20200625       1
20210810       1
20180709       1
20190707       1
20210422       1
20210322       1
20190324       1
20210928       1
20190106       1
20210524       1
20210126       1
20190516       1
20220215       1
20191028       1
20190208       1
20190820       1
20220329       1
20200901       1
20200525       1
20220306       1
20200121      

In [12]:
# df_레이아웃 / df_코드집 / df_co_재무계정 / df_재무데이터 / df_액티브중소법인재무_기업개요외감 /
# df_액티브중소법인재무_기업개요 / df_액티브중소법인재무_휴폐업이력 / df_휴폐업중소법인재무_기업개요 / df_휴폐업중소법인재무_휴폐업이력
print(len(df_코드집))
print(len(df_co_재무계정))
# print(len(df_재무데이터))
print(len(df_액티브중소법인재무_기업개요외감))

print(len(df_액티브중소법인재무_기업개요))
print(len(df_액티브중소법인재무_휴폐업이력))
print(len(df_휴폐업중소법인재무_기업개요))
print(len(df_휴폐업중소법인재무_휴폐업이력))

43
2055
28982
198651
4827
6739
6840


In [13]:
print(df_레이아웃.columns)
print(df_코드집.columns)
print(df_co_재무계정.columns)
# print(df_재무데이터.columns)

print(df_액티브중소법인재무_기업개요외감.columns)
print(df_액티브중소법인재무_기업개요.columns)
print(df_액티브중소법인재무_휴폐업이력.columns)
print(df_휴폐업중소법인재무_기업개요.columns)
print(df_휴폐업중소법인재무_휴폐업이력.columns)

Index(['순서', '테이블명', '영문칼럼명', '한글칼럼명', '칼럼형식', '디폴트값', 'NULL여부', '비고'], dtype='object')
Index(['GRP_CD', 'CD', 'END_DATE', 'STRT_DATE', 'CD_NM'], dtype='object')
Index(['계정코드', '순번', '계정명', '영문'], dtype='object')
Index(['BIZ_NO', 'CMP_PFIX_NM', 'CMP_NM1', 'CMP_SFIX_NM', 'CMP_ENM', 'BZ_TYP',
       'CMP_SCL', 'PBCO_GB', 'PSN_CORP_GB', 'HDOF_BR_GB', 'FR_IVST_CORP_YN',
       'VENT_YN', 'LIST_CD', 'IND_CD_ORDR', 'IND_CD1', 'IND_CD2', 'IND_CD3',
       'PB_ORG_TYP', 'MDSCO_PRTC_YN', 'HDOF_CMP_CD', 'ESTB_DATE'],
      dtype='object')
Index(['BIZ_NO', 'CMP_PFIX_NM', 'CMP_NM', 'CMP_NM1', 'CMP_SFIX_NM', 'CMP_ENM',
       'BZ_TYP', 'CMP_TYP', 'CMP_SCL', 'PBCO_GB', 'PSN_CORP_GB', 'HDOF_BR_GB',
       'FR_IVST_CORP_YN', 'VENT_YN', 'LIST_CD', 'IND_CD_ORDR', 'IND_CD1',
       'IND_CD2', 'IND_CD3', 'PB_ORG_TYP', 'MDSCO_PRTC_YN', 'TAXN_TYP',
       'HDOF_CMP_CD', 'ESTB_DATE', 'ESTB_GB', 'LIST_DATE', 'MN_BIZ_CONT',
       'NATN_NM', 'HOMEPAGE_URL', 'CEO_NM', 'EMP_CNT'],
      dtype='object')
Index(['B

In [14]:
# df_휴폐업중소법인재무["BIZ_NO"].unique() == df_액티브중소법인재무["BIZ_NO"].unique()
# df_액티브중소법인재무_기업개요외감["BIZ_NO"].unique() == df_휴폐업중소법인재무_휴폐업이력["BIZ_NO"].unique()
df_휴폐업중소법인재무_기업개요["BIZ_NO"].unique() == df_휴폐업중소법인재무_휴폐업이력["BIZ_NO"].unique()   # 같지않음 

False

In [15]:
df_액티브중소법인재무_휴폐업이력['CLSBZ_GB'].value_counts()
# 1 > 부도 / 2> 휴업 / 3> 폐업 / 4> 피합병 / 5> 휴업취소 / 6> 폐업취소 

6    1264
3    1239
2    1168
5    1155
4       1
Name: CLSBZ_GB, dtype: int64

In [16]:
df_휴폐업중소법인재무_휴폐업이력['CLSBZ_GB'].value_counts()

3    6838
5       1
6       1
Name: CLSBZ_GB, dtype: int64

In [17]:
df_액티브중소법인재무_기업개요.head()

,BIZ_NO,CMP_PFIX_NM,CMP_NM,CMP_NM1,CMP_SFIX_NM,CMP_ENM,BZ_TYP,CMP_TYP,CMP_SCL,PBCO_GB,PSN_CORP_GB,HDOF_BR_GB,FR_IVST_CORP_YN,VENT_YN,LIST_CD,IND_CD_ORDR,IND_CD1,IND_CD2,IND_CD3,PB_ORG_TYP,MDSCO_PRTC_YN,TAXN_TYP,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,MN_BIZ_CONT,NATN_NM,HOMEPAGE_URL,CEO_NM,EMP_CNT
0,2218116923,NaN,NaN,신우종합건설,(주),"Shinu General Construction Co., Ltd.",M,4,2,2,1,1,N,N,NaN,10.0,41200.0,NaN,NaN,0,N,3.0,NaN,20000807.0,1.0,NaN,토목공사업,NaN,NaN,김주대외 1명,12.0
1,1138111739,NaN,한일가스산업(주),한일가스산업,(주),"HANIL GAS IND. CO.,LTD.",M,3,2,2,1,1,N,N,NaN,10.0,47712.0,NaN,NaN,0,N,3.0,NaN,19660419.0,1.0,NaN,LP Gas 충전 및 판매업,NaN,www.hanilgas.com,홍순철외 1명,43.0
2,1108146490,(주),NaN,랩솔루션,NaN,Lab Solution,M,4,2,2,1,1,N,N,NaN,10.0,27210.0,27213.0,NaN,0,N,3.0,NaN,20000612.0,1.0,NaN,기체분석기 등,NaN,www.lab-solution.com,김봉철,3.0
3,3128134737,(주),NaN,고려인삼제품공사,NaN,"KOREAN GINSENG PRODUCT CO., LTD",M,4,2,2,1,1,N,N,NaN,10.0,10795.0,NaN,NaN,0,N,3.0,NaN,19920301.0,1.0,NaN,"다류, 차, 인삼제품, 음료, 코코아가공품, 커피",NaN,www.kitea.co.kr,박경출,24.0
4,2078132193,(주),(주)엠피아이,엠피아이,NaN,"EMPIALEE CO.,LTD.",M,3,2,2,1,1,N,N,NaN,10.0,15190.0,NaN,NaN,0,N,3.0,NaN,19910806.0,1.0,NaN,가죽 및 대용가죽제품 제조업,NaN,www.mastercnd.co.kr,이스테판상수외 1명,35.0


In [18]:
[df_액티브중소법인재무_기업개요외감['BIZ_NO']]

[0        1138111739
 1        2078132193
 2        1318167565
 3        5048128251
 4        1098164260
             ...    
 28977    8048101453
 28978    8028701430
 28979    4268501421
 28980    6518502462
 28981    6698502065
 Name: BIZ_NO, Length: 28982, dtype: int64]

# 순액티브, 순휴폐업 df 만들기

In [19]:
# df_액티브중소법인재무_기업개요외감["BIZ_NO"] 
# 순액티브 만들어주기 >> 기업개요외감 테이블만 쓸 것이므로 biz_no 기준,, 액티브_기업외감과 액티브_휴폐업이력 시트가 일치하지 않은 데이터는 날리기!   
for i in range(len(df_액티브중소법인재무_휴폐업이력)):
    df_액티브중소법인재무_기업개요외감 = df_액티브중소법인재무_기업개요외감[df_액티브중소법인재무_기업개요외감['BIZ_NO'] != df_액티브중소법인재무_휴폐업이력['BIZ_NO'][i]] 


In [20]:
# 순휴페업 df 만들기 
# df_휴폐업중소법인재무_휴폐업이력 = df_휴폐업중소법인재무_휴폐업이력.drop_duplicates(['BIZ_NO'])

In [21]:
len(df_휴폐업중소법인재무_기업개요)
len(df_휴폐업중소법인재무_휴폐업이력) # 휴폐업 기업개요랑 7개 차이가 남 >> merge 후 누락시킬 것.. . .ㅎㅎ 

6840

In [22]:
df_순휴폐업 = df_휴폐업중소법인재무_휴폐업이력.merge(df_휴폐업중소법인재무_기업개요, how = 'left', left_on = 'BIZ_NO', right_on = 'BIZ_NO')
# 날리기 완료 

In [23]:
len(df_액티브중소법인재무_기업개요외감) # 이게 순액티브 

28620

In [24]:
# df_액티브중소법인재무_기업개요외감 > 순액티브고 // df_순휴폐업 
# 순액티브에서 CMP_NM1 이름 바꾸고 순휴폐업이랑 컬럼명 맞추기 
df_액티브중소법인재무_기업개요외감.rename(columns={'CMP_NM1':'CMP_NM'})

,BIZ_NO,CMP_PFIX_NM,CMP_NM,CMP_SFIX_NM,CMP_ENM,BZ_TYP,CMP_SCL,PBCO_GB,PSN_CORP_GB,HDOF_BR_GB,FR_IVST_CORP_YN,VENT_YN,LIST_CD,IND_CD_ORDR,IND_CD1,IND_CD2,IND_CD3,PB_ORG_TYP,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE
0,1138111739,NaN,한일가스산업,(주),"HANIL GAS IND. CO.,LTD.",M,2,2,1,1,N,N,NaN,10.0,47712.0,NaN,NaN,0,N,NaN,19660419.0
1,2078132193,(주),엠피아이,NaN,"EMPIALEE CO.,LTD.",M,2,2,1,1,N,N,NaN,10.0,15190.0,NaN,NaN,0,N,NaN,19910806.0
2,1318167565,NaN,도영운수,(주),"DOYOUNG TRANSPORTATION CO.,LTD.",M,2,2,1,1,N,N,NaN,10.0,49212.0,NaN,NaN,0,N,NaN,20011030.0
3,5048128251,(주),중원산업,NaN,"JOONGWON INDUSTRIAL CO.,LTD.",M,2,2,1,1,N,N,NaN,10.0,30391.0,NaN,NaN,0,N,NaN,19990112.0
4,1098164260,(주),카타나골프,NaN,"CATANA GOLF CO.,LTD.",M,2,2,1,1,N,N,NaN,10.0,46800.0,NaN,NaN,0,N,NaN,20000531.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28977,8048101453,NaN,제이앤인베스트먼트,NaN,J&INVESTMENT,M,2,2,1,1,N,N,NaN,NaN,NaN,NaN,NaN,0,N,NaN,20191206.0
28978,8028701430,NaN,디앤엠개발,(주),"DNM Development Co., Ltd.",M,2,2,1,1,N,N,NaN,10.0,68122.0,NaN,NaN,0,N,NaN,20200402.0
28979,4268501421,NaN,마스비 건축연구소,NaN,NaN,M,2,2,1,2,N,N,NaN,10.0,72100.0,NaN,NaN,0,N,1220726.0,19900108.0
28980,6518502462,(주),건설레미콘의정부공장,NaN,"Construction Remicon Uijeongbu Factory Co., Ltd.",M,2,2,1,2,N,N,NaN,10.0,23322.0,NaN,NaN,0,N,1583116.0,20220615.0


In [25]:
df_순휴폐업.head()

,BIZ_NO,END_DATE,STRT_DATE,CLSBZ_GB,STAT_OCR_DATE,CMP_PFIX_NM,CMP_NM,CMP_SFIX_NM,CMP_ENM,BZ_TYP,CMP_SCL,PBCO_GB,PSN_CORP_GB,HDOF_BR_GB,FR_IVST_CORP_YN,VENT_YN,LIST_CD,IND_CD_ORDR,IND_CD1,IND_CD2,IND_CD3,PB_ORG_TYP,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,MN_BIZ_CONT,NATN_NM,HOMEPAGE_URL,CEO_NM,EMP_CNT
0,1268145633,99991231,20220518,3,20220518,(주),용진테크,NaN,"Yongjin Tech Co., Ltd.",M,2.0,2.0,1.0,1.0,N,N,NaN,10.0,30391.0,NaN,NaN,0.0,N,NaN,20000529.0,1.0,NaN,자동차부품 제조판매업,NaN,www.yongjintech.com,안상동,40.0
1,1388101749,99991231,20210319,3,20210319,NaN,신구건설,(주),"SHINKU C0NSTRUCTION CO.,LTD.",M,2.0,2.0,1.0,1.0,N,N,NaN,10.0,46610.0,NaN,NaN,0.0,N,NaN,19850110.0,1.0,NaN,건축자재 판매업,NaN,www.shinku.co.kr,소재필,13.0
2,1068179240,99991231,20200731,3,20191213,(주),석정미디어,NaN,sung jung,M,2.0,2.0,1.0,1.0,N,N,NaN,10.0,46510.0,NaN,NaN,0.0,N,NaN,19990601.0,1.0,NaN,"토너, 잉크, 프린터 외 컴퓨터 관련 주변기기",NaN,NaN,이상호,1.0
3,2248114507,99991231,20210531,3,20210531,(주),새한실업,NaN,"SAEHAN INDUSTRY CO.,LTD.",M,2.0,2.0,1.0,1.0,N,N,NaN,10.0,25991.0,NaN,NaN,0.0,N,NaN,19980522.0,1.0,NaN,금속 캔 및 기타 포장용기 제조업,NaN,NaN,김정남외 1명,6.0
4,1168132785,99991231,20210630,3,20210630,(주),석영엔터프라이즈,NaN,"Seokyoung Enterprise Co.,Ltd.",M,2.0,2.0,1.0,1.0,N,N,NaN,10.0,24199.0,NaN,NaN,0.0,N,NaN,19930226.0,1.0,NaN,스마트트레이 외,NaN,www.systeel.co.kr,박성진,40.0


In [26]:
df_순휴폐업.info()  # 순휴폐업 6739개의 기업 
df_액티브중소법인재무_기업개요외감.info() # 순액티브 28620개의 기업

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6840 entries, 0 to 6839
Data columns (total 32 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   BIZ_NO           6840 non-null   int64  
 1   END_DATE         6840 non-null   int64  
 2   STRT_DATE        6840 non-null   int64  
 3   CLSBZ_GB         6840 non-null   int64  
 4   STAT_OCR_DATE    6840 non-null   int64  
 5   CMP_PFIX_NM      4915 non-null   object 
 6   CMP_NM           6832 non-null   object 
 7   CMP_SFIX_NM      1302 non-null   object 
 8   CMP_ENM          6788 non-null   object 
 9   BZ_TYP           6832 non-null   object 
 10  CMP_SCL          6832 non-null   float64
 11  PBCO_GB          6832 non-null   float64
 12  PSN_CORP_GB      6832 non-null   float64
 13  HDOF_BR_GB       6832 non-null   float64
 14  FR_IVST_CORP_YN  6832 non-null   object 
 15  VENT_YN          6832 non-null   object 
 16  LIST_CD          46 non-null     object 
 17  IND_CD_ORDR   

In [27]:
df_순휴폐업.isna().sum()

BIZ_NO                0
END_DATE              0
STRT_DATE             0
CLSBZ_GB              0
STAT_OCR_DATE         0
CMP_PFIX_NM        1925
CMP_NM                8
CMP_SFIX_NM        5538
CMP_ENM              52
BZ_TYP                8
CMP_SCL               8
PBCO_GB               8
PSN_CORP_GB           8
HDOF_BR_GB            8
FR_IVST_CORP_YN       8
VENT_YN               8
LIST_CD            6794
IND_CD_ORDR          13
IND_CD1             162
IND_CD2            6757
IND_CD3            6829
PB_ORG_TYP            8
MDSCO_PRTC_YN         8
HDOF_CMP_CD        5374
ESTB_DATE           577
ESTB_GB               8
LIST_DATE          6791
MN_BIZ_CONT         179
NATN_NM            6696
HOMEPAGE_URL       4686
CEO_NM                9
EMP_CNT             705
dtype: int64

In [28]:
df_액티브중소법인재무_기업개요외감.isna().sum()

BIZ_NO                 0
CMP_PFIX_NM        10295
CMP_NM1                0
CMP_SFIX_NM        19550
CMP_ENM               17
BZ_TYP                 0
CMP_SCL                0
PBCO_GB                0
PSN_CORP_GB            0
HDOF_BR_GB             0
FR_IVST_CORP_YN        0
VENT_YN                0
LIST_CD            28531
IND_CD_ORDR           10
IND_CD1               11
IND_CD2            27424
IND_CD3            28358
PB_ORG_TYP             0
MDSCO_PRTC_YN          0
HDOF_CMP_CD        25745
ESTB_DATE           1059
dtype: int64

In [29]:
df_순휴폐업['CLSBZ_GB'].value_counts() # 참고로 피합병은 부도라고 보기 어렵기 때문에 CLSBZ_GB ==4인 기업은 제외,, 

3    6838
5       1
6       1
Name: CLSBZ_GB, dtype: int64

In [30]:
df_순휴폐업.columns

Index(['BIZ_NO', 'END_DATE', 'STRT_DATE', 'CLSBZ_GB', 'STAT_OCR_DATE',
       'CMP_PFIX_NM', 'CMP_NM', 'CMP_SFIX_NM', 'CMP_ENM', 'BZ_TYP', 'CMP_SCL',
       'PBCO_GB', 'PSN_CORP_GB', 'HDOF_BR_GB', 'FR_IVST_CORP_YN', 'VENT_YN',
       'LIST_CD', 'IND_CD_ORDR', 'IND_CD1', 'IND_CD2', 'IND_CD3', 'PB_ORG_TYP',
       'MDSCO_PRTC_YN', 'HDOF_CMP_CD', 'ESTB_DATE', 'ESTB_GB', 'LIST_DATE',
       'MN_BIZ_CONT', 'NATN_NM', 'HOMEPAGE_URL', 'CEO_NM', 'EMP_CNT'],
      dtype='object')

# 부도여부 타겟값 추가 

In [31]:
# 부도여부 타겟값 추가 
df_순휴폐업['부도'] = 1 
df_액티브중소법인재무_기업개요외감['부도'] = 0

In [32]:
df_순휴폐업['부도'].value_counts()
df_액티브중소법인재무_기업개요외감['부도'].value_counts()

0    28620
Name: 부도, dtype: int64

In [33]:
df_순휴폐업.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6840 entries, 0 to 6839
Data columns (total 33 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   BIZ_NO           6840 non-null   int64  
 1   END_DATE         6840 non-null   int64  
 2   STRT_DATE        6840 non-null   int64  
 3   CLSBZ_GB         6840 non-null   int64  
 4   STAT_OCR_DATE    6840 non-null   int64  
 5   CMP_PFIX_NM      4915 non-null   object 
 6   CMP_NM           6832 non-null   object 
 7   CMP_SFIX_NM      1302 non-null   object 
 8   CMP_ENM          6788 non-null   object 
 9   BZ_TYP           6832 non-null   object 
 10  CMP_SCL          6832 non-null   float64
 11  PBCO_GB          6832 non-null   float64
 12  PSN_CORP_GB      6832 non-null   float64
 13  HDOF_BR_GB       6832 non-null   float64
 14  FR_IVST_CORP_YN  6832 non-null   object 
 15  VENT_YN          6832 non-null   object 
 16  LIST_CD          46 non-null     object 
 17  IND_CD_ORDR   

In [34]:
# 이건 필요없지만 혹시 몰라서 남겨놓음 

# df_다합친파일_re = pd.concat([ df_순휴폐업 , df_액티브중소법인재무_기업개요외감 ] , axis = 0)
# df_다합친파일_re.head()
# len(df_다합친파일_re)

# df_다합친파일_re.to_excel("./df_다합친파일.xlsx", index=False)


In [35]:
estb_fillna = pd.read_csv('/content/drive/MyDrive/nice공모전/ESTB_2.csv')

In [36]:
estb_fillna

,BIZ_NO,CMP_NM1
0,1258504073,19570528.0
1,1238519395,19950721.0
2,1238531860,20220111.0
3,6198501815,19950621.0
4,1168500184,19550513.0
...,...,...
1069,5488500834,19720202.0
1070,8758501021,20141226.0
1071,6578501234,20050101.0
1072,1098544689,20080415.0


In [37]:
df_액티브중소법인재무_기업개요외감.head()

,BIZ_NO,CMP_PFIX_NM,CMP_NM1,CMP_SFIX_NM,CMP_ENM,BZ_TYP,CMP_SCL,PBCO_GB,PSN_CORP_GB,HDOF_BR_GB,FR_IVST_CORP_YN,VENT_YN,LIST_CD,IND_CD_ORDR,IND_CD1,IND_CD2,IND_CD3,PB_ORG_TYP,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,부도
0,1138111739,NaN,한일가스산업,(주),"HANIL GAS IND. CO.,LTD.",M,2,2,1,1,N,N,NaN,10.0,47712.0,NaN,NaN,0,N,NaN,19660419.0,0
1,2078132193,(주),엠피아이,NaN,"EMPIALEE CO.,LTD.",M,2,2,1,1,N,N,NaN,10.0,15190.0,NaN,NaN,0,N,NaN,19910806.0,0
2,1318167565,NaN,도영운수,(주),"DOYOUNG TRANSPORTATION CO.,LTD.",M,2,2,1,1,N,N,NaN,10.0,49212.0,NaN,NaN,0,N,NaN,20011030.0,0
3,5048128251,(주),중원산업,NaN,"JOONGWON INDUSTRIAL CO.,LTD.",M,2,2,1,1,N,N,NaN,10.0,30391.0,NaN,NaN,0,N,NaN,19990112.0,0
4,1098164260,(주),카타나골프,NaN,"CATANA GOLF CO.,LTD.",M,2,2,1,1,N,N,NaN,10.0,46800.0,NaN,NaN,0,N,NaN,20000531.0,0


In [38]:
estb_fillna = estb_fillna.rename(columns={'CMP_NM1':'ESTB_DATE'})


In [39]:
df_액티브중소법인재무_기업개요외감['ESTB_DATE']

0        19660419.0
1        19910806.0
2        20011030.0
3        19990112.0
4        20000531.0
            ...    
28977    20191206.0
28978    20200402.0
28979    19900108.0
28980    20220615.0
28981    20211209.0
Name: ESTB_DATE, Length: 28620, dtype: float64

In [40]:
df_액티브중소법인재무_기업개요외감['ESTB_DATE']

0        19660419.0
1        19910806.0
2        20011030.0
3        19990112.0
4        20000531.0
            ...    
28977    20191206.0
28978    20200402.0
28979    19900108.0
28980    20220615.0
28981    20211209.0
Name: ESTB_DATE, Length: 28620, dtype: float64

In [41]:
df_액티브중소법인재무_기업개요외감.columns

Index(['BIZ_NO', 'CMP_PFIX_NM', 'CMP_NM1', 'CMP_SFIX_NM', 'CMP_ENM', 'BZ_TYP',
       'CMP_SCL', 'PBCO_GB', 'PSN_CORP_GB', 'HDOF_BR_GB', 'FR_IVST_CORP_YN',
       'VENT_YN', 'LIST_CD', 'IND_CD_ORDR', 'IND_CD1', 'IND_CD2', 'IND_CD3',
       'PB_ORG_TYP', 'MDSCO_PRTC_YN', 'HDOF_CMP_CD', 'ESTB_DATE', '부도'],
      dtype='object')

In [42]:
df_액티브중소법인재무_기업개요외감= df_액티브중소법인재무_기업개요외감.merge(estb_fillna, how = 'left', left_on = 'BIZ_NO', right_on = 'BIZ_NO')

In [43]:
df_액티브중소법인재무_기업개요외감[df_액티브중소법인재무_기업개요외감['ESTB_DATE_y'].isna()]

,BIZ_NO,CMP_PFIX_NM,CMP_NM1,CMP_SFIX_NM,CMP_ENM,BZ_TYP,CMP_SCL,PBCO_GB,PSN_CORP_GB,HDOF_BR_GB,FR_IVST_CORP_YN,VENT_YN,LIST_CD,IND_CD_ORDR,IND_CD1,IND_CD2,IND_CD3,PB_ORG_TYP,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE_x,부도,ESTB_DATE_y
0,1138111739,NaN,한일가스산업,(주),"HANIL GAS IND. CO.,LTD.",M,2,2,1,1,N,N,NaN,10.0,47712.0,NaN,NaN,0,N,NaN,19660419.0,0,NaN
1,2078132193,(주),엠피아이,NaN,"EMPIALEE CO.,LTD.",M,2,2,1,1,N,N,NaN,10.0,15190.0,NaN,NaN,0,N,NaN,19910806.0,0,NaN
2,1318167565,NaN,도영운수,(주),"DOYOUNG TRANSPORTATION CO.,LTD.",M,2,2,1,1,N,N,NaN,10.0,49212.0,NaN,NaN,0,N,NaN,20011030.0,0,NaN
3,5048128251,(주),중원산업,NaN,"JOONGWON INDUSTRIAL CO.,LTD.",M,2,2,1,1,N,N,NaN,10.0,30391.0,NaN,NaN,0,N,NaN,19990112.0,0,NaN
4,1098164260,(주),카타나골프,NaN,"CATANA GOLF CO.,LTD.",M,2,2,1,1,N,N,NaN,10.0,46800.0,NaN,NaN,0,N,NaN,20000531.0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28630,8048101453,NaN,제이앤인베스트먼트,NaN,J&INVESTMENT,M,2,2,1,1,N,N,NaN,NaN,NaN,NaN,NaN,0,N,NaN,20191206.0,0,NaN
28631,8028701430,NaN,디앤엠개발,(주),"DNM Development Co., Ltd.",M,2,2,1,1,N,N,NaN,10.0,68122.0,NaN,NaN,0,N,NaN,20200402.0,0,NaN
28632,4268501421,NaN,마스비 건축연구소,NaN,NaN,M,2,2,1,2,N,N,NaN,10.0,72100.0,NaN,NaN,0,N,1220726.0,19900108.0,0,NaN
28633,6518502462,(주),건설레미콘의정부공장,NaN,"Construction Remicon Uijeongbu Factory Co., Ltd.",M,2,2,1,2,N,N,NaN,10.0,23322.0,NaN,NaN,0,N,1583116.0,20220615.0,0,NaN


In [44]:
df_액티브중소법인재무_기업개요외감['ESTB_DATE_y'].isna().sum()
# df_액티브중소법인재무_기업개요외감['ESTB_DATE_x'].isna().sum()


27581

In [45]:
df_액티브중소법인재무_기업개요외감

,BIZ_NO,CMP_PFIX_NM,CMP_NM1,CMP_SFIX_NM,CMP_ENM,BZ_TYP,CMP_SCL,PBCO_GB,PSN_CORP_GB,HDOF_BR_GB,FR_IVST_CORP_YN,VENT_YN,LIST_CD,IND_CD_ORDR,IND_CD1,IND_CD2,IND_CD3,PB_ORG_TYP,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE_x,부도,ESTB_DATE_y
0,1138111739,NaN,한일가스산업,(주),"HANIL GAS IND. CO.,LTD.",M,2,2,1,1,N,N,NaN,10.0,47712.0,NaN,NaN,0,N,NaN,19660419.0,0,NaN
1,2078132193,(주),엠피아이,NaN,"EMPIALEE CO.,LTD.",M,2,2,1,1,N,N,NaN,10.0,15190.0,NaN,NaN,0,N,NaN,19910806.0,0,NaN
2,1318167565,NaN,도영운수,(주),"DOYOUNG TRANSPORTATION CO.,LTD.",M,2,2,1,1,N,N,NaN,10.0,49212.0,NaN,NaN,0,N,NaN,20011030.0,0,NaN
3,5048128251,(주),중원산업,NaN,"JOONGWON INDUSTRIAL CO.,LTD.",M,2,2,1,1,N,N,NaN,10.0,30391.0,NaN,NaN,0,N,NaN,19990112.0,0,NaN
4,1098164260,(주),카타나골프,NaN,"CATANA GOLF CO.,LTD.",M,2,2,1,1,N,N,NaN,10.0,46800.0,NaN,NaN,0,N,NaN,20000531.0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28630,8048101453,NaN,제이앤인베스트먼트,NaN,J&INVESTMENT,M,2,2,1,1,N,N,NaN,NaN,NaN,NaN,NaN,0,N,NaN,20191206.0,0,NaN
28631,8028701430,NaN,디앤엠개발,(주),"DNM Development Co., Ltd.",M,2,2,1,1,N,N,NaN,10.0,68122.0,NaN,NaN,0,N,NaN,20200402.0,0,NaN
28632,4268501421,NaN,마스비 건축연구소,NaN,NaN,M,2,2,1,2,N,N,NaN,10.0,72100.0,NaN,NaN,0,N,1220726.0,19900108.0,0,NaN
28633,6518502462,(주),건설레미콘의정부공장,NaN,"Construction Remicon Uijeongbu Factory Co., Ltd.",M,2,2,1,2,N,N,NaN,10.0,23322.0,NaN,NaN,0,N,1583116.0,20220615.0,0,NaN


In [46]:
df_액티브중소법인재무_기업개요외감 = df_액티브중소법인재무_기업개요외감.rename(columns={'ESTB_DATE_y':'ESTB_DATE'})


# data split

In [47]:
from sklearn.model_selection import train_test_split
# 순휴폐업 train, test용 분리  
X = df_순휴폐업
순휴폐업_train , 순휴폐업_test = train_test_split(X, test_size=0.2, random_state=1111)
순휴폐업_train.shape, 순휴폐업_test.shape


((5472, 33), (1368, 33))

In [48]:
# 순액티브 train, test용 분리  
X = df_액티브중소법인재무_기업개요외감

순액티브_train , 순액티브_test = train_test_split(X, test_size=0.2, random_state=1111)
순액티브_train.shape, 순액티브_test.shape


((22908, 23), (5727, 23))

In [49]:
# train은 train끼리 test는 test끼리 합치기! 
휴폐업예측_train = pd.concat([ 순휴폐업_train , 순액티브_train ] , axis = 0 )
휴폐업예측_test = pd.concat([ 순휴폐업_test , 순액티브_test ] , axis = 0 )


In [50]:
휴폐업예측_train.shape , 휴폐업예측_test.shape

((28380, 35), (7095, 35))

In [51]:
휴폐업예측_train.head()

,BIZ_NO,END_DATE,STRT_DATE,CLSBZ_GB,STAT_OCR_DATE,CMP_PFIX_NM,CMP_NM,CMP_SFIX_NM,CMP_ENM,BZ_TYP,CMP_SCL,PBCO_GB,PSN_CORP_GB,HDOF_BR_GB,FR_IVST_CORP_YN,VENT_YN,LIST_CD,IND_CD_ORDR,IND_CD1,IND_CD2,IND_CD3,PB_ORG_TYP,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,MN_BIZ_CONT,NATN_NM,HOMEPAGE_URL,CEO_NM,EMP_CNT,부도,CMP_NM1,ESTB_DATE_x
6212,4488700862,99991231.0,20210923.0,3.0,20210923.0,(주),유준,NaN,"U Jun Co., Ltd.",M,2.0,2.0,1.0,1.0,N,N,NaN,10.0,46499.0,NaN,NaN,0.0,N,NaN,20180108.0,1.0,NaN,잡화 도매,NaN,NaN,우대원,0.0,1,NaN,NaN
4233,1428700101,99991231.0,20210930.0,3.0,20210930.0,(주),세상,NaN,"World Co., Ltd.",M,2.0,2.0,1.0,1.0,N,N,NaN,10.0,46311.0,NaN,NaN,0.0,N,NaN,20150216.0,1.0,NaN,"농산물 도,소매,유통,판매업",NaN,NaN,성광자,4.0,1,NaN,NaN
1055,4028158497,99991231.0,20200925.0,3.0,20200925.0,NaN,고려상조,(주),"KOREASANGJO CO.,LTD.",M,2.0,2.0,1.0,1.0,N,N,NaN,10.0,96921.0,NaN,NaN,0.0,N,NaN,20040322.0,1.0,NaN,장의 용영업-서비스,NaN,www.koreasangjo.net,황병태,5.0,1,NaN,NaN
375,6088123771,99991231.0,20220531.0,3.0,20220531.0,NaN,삼영설비,(주),"Samyoung Equipment Co., Ltd.",M,2.0,2.0,1.0,1.0,N,N,NaN,10.0,42201.0,NaN,NaN,0.0,N,NaN,19951024.0,1.0,NaN,"기계설비공사, 소방시설공사",NaN,NaN,김재철,18.0,1,NaN,NaN
6614,3478501093,99991231.0,20200731.0,3.0,20200601.0,(주),탑인터큐브,NaN,"Top Inte Cube Co., Ltd.",M,2.0,2.0,1.0,2.0,N,N,NaN,10.0,46109.0,NaN,NaN,0.0,N,2987547.0,20190201.0,1.0,NaN,"수출입업및동대행업, 무역관련 외 도매 및 소매업 외",NaN,NaN,이운용,30.0,1,NaN,NaN


In [52]:
휴폐업예측_train['부도'].value_counts()

0    22908
1     5472
Name: 부도, dtype: int64

In [53]:
휴폐업예측_test['부도'].value_counts()

0    5727
1    1368
Name: 부도, dtype: int64

In [54]:
휴폐업예측_train.columns

Index(['BIZ_NO', 'END_DATE', 'STRT_DATE', 'CLSBZ_GB', 'STAT_OCR_DATE',
       'CMP_PFIX_NM', 'CMP_NM', 'CMP_SFIX_NM', 'CMP_ENM', 'BZ_TYP', 'CMP_SCL',
       'PBCO_GB', 'PSN_CORP_GB', 'HDOF_BR_GB', 'FR_IVST_CORP_YN', 'VENT_YN',
       'LIST_CD', 'IND_CD_ORDR', 'IND_CD1', 'IND_CD2', 'IND_CD3', 'PB_ORG_TYP',
       'MDSCO_PRTC_YN', 'HDOF_CMP_CD', 'ESTB_DATE', 'ESTB_GB', 'LIST_DATE',
       'MN_BIZ_CONT', 'NATN_NM', 'HOMEPAGE_URL', 'CEO_NM', 'EMP_CNT', '부도',
       'CMP_NM1', 'ESTB_DATE_x'],
      dtype='object')

# 홈페이지주소유무

In [55]:
# HOMEPAGE_URL 이상한 값들이 넣어져있는건 0으로 대체해주기 

휴폐업예측_train["HOMEPAGE_URL"] = 휴폐업예측_train["HOMEPAGE_URL"].replace( "없음", 0)
휴폐업예측_train["HOMEPAGE_URL"] = 휴폐업예측_train["HOMEPAGE_URL"].replace( "-", 0)
휴폐업예측_train["HOMEPAGE_URL"] = 휴폐업예측_train["HOMEPAGE_URL"].replace( "--", 0)
휴폐업예측_train["HOMEPAGE_URL"] = 휴폐업예측_train["HOMEPAGE_URL"].replace( "_", 0)
휴폐업예측_train["HOMEPAGE_URL"] = 휴폐업예측_train["HOMEPAGE_URL"].replace( "없슴", 0)
휴폐업예측_train["HOMEPAGE_URL"] = 휴폐업예측_train["HOMEPAGE_URL"].replace( "없음.", 0)
휴폐업예측_train["HOMEPAGE_URL"] = 휴폐업예측_train["HOMEPAGE_URL"].replace( ".", 0)
휴폐업예측_train["HOMEPAGE_URL"] = 휴폐업예측_train["HOMEPAGE_URL"].replace( "nan", 0)
휴폐업예측_train["HOMEPAGE_URL"] = 휴폐업예측_train["HOMEPAGE_URL"].replace( "무", 0)
휴폐업예측_train["HOMEPAGE_URL"] = 휴폐업예측_train["HOMEPAGE_URL"].replace( "해당사항 없음", 0)
휴폐업예측_train["HOMEPAGE_URL"] = 휴폐업예측_train["HOMEPAGE_URL"].replace( "해당사항없슴", 0)
휴폐업예측_train["HOMEPAGE_URL"] = 휴폐업예측_train["HOMEPAGE_URL"].replace( "해당사항없음", 0)
휴폐업예측_train["HOMEPAGE_URL"] = 휴폐업예측_train["HOMEPAGE_URL"].replace( "해당없슴", 0)
휴폐업예측_train["HOMEPAGE_URL"] = 휴폐업예측_train["HOMEPAGE_URL"].replace( "na", 0)
휴폐업예측_train["HOMEPAGE_URL"] = 휴폐업예측_train["HOMEPAGE_URL"].replace( "n/a", 0)
휴폐업예측_train["HOMEPAGE_URL"] = 휴폐업예측_train["HOMEPAGE_URL"].replace( "x", 0)
휴폐업예측_train["HOMEPAGE_URL"] = 휴폐업예측_train["HOMEPAGE_URL"].replace( "www", 0)


휴폐업예측_train["HOMEPAGE_URL"] = 휴폐업예측_train["HOMEPAGE_URL"].fillna(0)


휴폐업예측_test["HOMEPAGE_URL"] = 휴폐업예측_test["HOMEPAGE_URL"].replace( "없음", 0)
휴폐업예측_test["HOMEPAGE_URL"] = 휴폐업예측_test["HOMEPAGE_URL"].replace( "-", 0)
휴폐업예측_test["HOMEPAGE_URL"] = 휴폐업예측_test["HOMEPAGE_URL"].replace( "--", 0)
휴폐업예측_test["HOMEPAGE_URL"] = 휴폐업예측_test["HOMEPAGE_URL"].replace( "_", 0)
휴폐업예측_test["HOMEPAGE_URL"] = 휴폐업예측_test["HOMEPAGE_URL"].replace( "없슴", 0)
휴폐업예측_test["HOMEPAGE_URL"] = 휴폐업예측_test["HOMEPAGE_URL"].replace( "없음.", 0)
휴폐업예측_test["HOMEPAGE_URL"] = 휴폐업예측_test["HOMEPAGE_URL"].replace( ".", 0)
휴폐업예측_test["HOMEPAGE_URL"] = 휴폐업예측_test["HOMEPAGE_URL"].replace( "nan", 0)
휴폐업예측_test["HOMEPAGE_URL"] = 휴폐업예측_test["HOMEPAGE_URL"].replace( "무", 0)
휴폐업예측_test["HOMEPAGE_URL"] = 휴폐업예측_test["HOMEPAGE_URL"].replace( "해당사항 없음", 0)
휴폐업예측_test["HOMEPAGE_URL"] = 휴폐업예측_test["HOMEPAGE_URL"].replace( "해당사항없슴", 0)
휴폐업예측_test["HOMEPAGE_URL"] = 휴폐업예측_test["HOMEPAGE_URL"].replace( "해당사항없음", 0)
휴폐업예측_test["HOMEPAGE_URL"] = 휴폐업예측_test["HOMEPAGE_URL"].replace( "해당없슴", 0)
휴폐업예측_test["HOMEPAGE_URL"] = 휴폐업예측_test["HOMEPAGE_URL"].replace( "na", 0)
휴폐업예측_test["HOMEPAGE_URL"] = 휴폐업예측_test["HOMEPAGE_URL"].replace( "n/a", 0)
휴폐업예측_test["HOMEPAGE_URL"] = 휴폐업예측_test["HOMEPAGE_URL"].replace( "x", 0)
휴폐업예측_test["HOMEPAGE_URL"] = 휴폐업예측_test["HOMEPAGE_URL"].replace( "www", 0)

휴폐업예측_test["HOMEPAGE_URL"] = 휴폐업예측_test["HOMEPAGE_URL"].fillna(0)

In [56]:
# @ 포함된 애들 다 바꾸기... 개인이메일이라..ㅎㅎ naver.com / hanmail.net 이런것들 .. .
# 휴폐업예측_train["HOMEPAGE_URL"] = 휴폐업예측_train["HOMEPAGE_URL"].replace( 휴폐업예측_train["HOMEPAGE_URL"].str.contains('naver'), 0)
# 휴폐업예측_train[휴폐업예측_train["HOMEPAGE_URL"].str.contains('@')]

휴폐업예측_train["HOMEPAGE_URL"] = np.where(휴폐업예측_train["HOMEPAGE_URL"].str.contains("@") , 0 , 휴폐업예측_train["HOMEPAGE_URL"])
휴폐업예측_test["HOMEPAGE_URL"] = np.where(휴폐업예측_test["HOMEPAGE_URL"].str.contains("@") , 0 , 휴폐업예측_test["HOMEPAGE_URL"])


In [57]:
# 휴폐업예측_train['HOMEPAGE_URL'].unique()
# 휴폐업예측_test['HOMEPAGE_URL'].unique()

In [58]:
# 1 아니면 0으로 다 바꿔버리기 
def change0not1(data):
    # if data != "":
    if data != 1:
        data = 0
        return data
    else:
        return data

In [59]:
휴폐업예측_train['HOMEPAGE_URL'].value_counts()

0                         26674
www.sci.co.kr                 5
www.mollis.co.kr              4
www.ildongfoodis.co.kr        4
www.e-chon.co.kr              3
                          ...  
www.dstour.com                1
www.fienva.co.kr              1
www.insideus.co.kr            1
www.makinochaya.co.kr         1
www.pntelecom.co.kr           1
Name: HOMEPAGE_URL, Length: 1628, dtype: int64

In [60]:
휴폐업예측_test['HOMEPAGE_URL'].value_counts()


0                         6705
www.daega24.co.kr            3
www.epostopia.com            2
www.kdeletronics.co.kr       1
www.enervis.co.kr            1
                          ... 
www.jwlgls.com               1
www.cniinfo.co.kr            1
www.flaktwoods.com           1
www.2xlub.com                1
www.jhwcake.com              1
Name: HOMEPAGE_URL, Length: 388, dtype: int64

In [61]:
# 0이 아니면 1로 바꿔버리기 
def change1not0(data):
    # if data != "":
    if data != 0:
        data = 1
        return data
    else:
        return data
휴폐업예측_train['HOMEPAGE_URL'] = 휴폐업예측_train['HOMEPAGE_URL'].apply(change1not0)
휴폐업예측_test['HOMEPAGE_URL'] = 휴폐업예측_test['HOMEPAGE_URL'].apply(change1not0)


In [62]:
휴폐업예측_train['HOMEPAGE_URL'] = 휴폐업예측_train['HOMEPAGE_URL'].apply(change1not0)
휴폐업예측_test['HOMEPAGE_URL'] = 휴폐업예측_test['HOMEPAGE_URL'].apply(change1not0)


In [63]:
len(휴폐업예측_train) # 28287
len(휴폐업예측_test)  # 7072
# 35359

7095

In [64]:
휴폐업예측_train['HOMEPAGE_URL'].value_counts()

# 0    26662
# 1     1706
# 홈페이지 없는 기업이 26662 ,, 있는 기업이 1706


0    26674
1     1706
Name: HOMEPAGE_URL, dtype: int64

In [65]:
# # 그냥 이대로 넣어서 점수내보기  __ train ^^ 
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import roc_auc_score

# y = 휴폐업예측_train['부도']
# X = 휴폐업예측_train.drop(['부도'],  axis=1)

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1111)

# lr_model1 = RandomForestClassifier()
# lr_model1.fit(X_train, y_train)

# lr_pred = lr_model.predict(X_test)
# roc_auc_score(y_test, lr_model1.predict_proba(X_test)[:, 1])


# 이걸 해보려고 했으나 생각해보니 object 너무 많아서 안됨... ㅎㅎ 정리부터 하고 낼 것 ! 


In [66]:
# 설립일 4글자로 잘라주기 (연도만!)
휴폐업예측_train['ESTB_DATE'] = 휴폐업예측_train['ESTB_DATE'].astype('str').str[:4]
휴폐업예측_test['ESTB_DATE'] = 휴폐업예측_test['ESTB_DATE'].astype('str').str[:4]

휴폐업예측_train['END_DATE'] = 휴폐업예측_train['END_DATE'].astype('str').str[:4]
휴폐업예측_test['END_DATE'] = 휴폐업예측_test['END_DATE'].astype('str').str[:4]


In [67]:
# 휴폐업예측_train['ESTB_DATE'] = 휴폐업예측_train['ESTB_DATE'].astype('int64')
# 휴폐업예측_test['ESTB_DATE'] = 휴폐업예측_test['ESTB_DATE'].astype('int64')
# 설립일에 nan이 있네 ^^ 하핫  (1302개나 있네)

In [68]:
휴폐업예측_train['END_DATE'].value_counts()
휴폐업예측_test['END_DATE'].value_counts()


nan     5727
9999    1346
2020       8
2022       5
2019       4
2021       4
2018       1
Name: END_DATE, dtype: int64

In [69]:
휴폐업예측_train.head()

,BIZ_NO,END_DATE,STRT_DATE,CLSBZ_GB,STAT_OCR_DATE,CMP_PFIX_NM,CMP_NM,CMP_SFIX_NM,CMP_ENM,BZ_TYP,CMP_SCL,PBCO_GB,PSN_CORP_GB,HDOF_BR_GB,FR_IVST_CORP_YN,VENT_YN,LIST_CD,IND_CD_ORDR,IND_CD1,IND_CD2,IND_CD3,PB_ORG_TYP,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,MN_BIZ_CONT,NATN_NM,HOMEPAGE_URL,CEO_NM,EMP_CNT,부도,CMP_NM1,ESTB_DATE_x
6212,4488700862,9999,20210923.0,3.0,20210923.0,(주),유준,NaN,"U Jun Co., Ltd.",M,2.0,2.0,1.0,1.0,N,N,NaN,10.0,46499.0,NaN,NaN,0.0,N,NaN,2018,1.0,NaN,잡화 도매,NaN,0,우대원,0.0,1,NaN,NaN
4233,1428700101,9999,20210930.0,3.0,20210930.0,(주),세상,NaN,"World Co., Ltd.",M,2.0,2.0,1.0,1.0,N,N,NaN,10.0,46311.0,NaN,NaN,0.0,N,NaN,2015,1.0,NaN,"농산물 도,소매,유통,판매업",NaN,0,성광자,4.0,1,NaN,NaN
1055,4028158497,9999,20200925.0,3.0,20200925.0,NaN,고려상조,(주),"KOREASANGJO CO.,LTD.",M,2.0,2.0,1.0,1.0,N,N,NaN,10.0,96921.0,NaN,NaN,0.0,N,NaN,2004,1.0,NaN,장의 용영업-서비스,NaN,1,황병태,5.0,1,NaN,NaN
375,6088123771,9999,20220531.0,3.0,20220531.0,NaN,삼영설비,(주),"Samyoung Equipment Co., Ltd.",M,2.0,2.0,1.0,1.0,N,N,NaN,10.0,42201.0,NaN,NaN,0.0,N,NaN,1995,1.0,NaN,"기계설비공사, 소방시설공사",NaN,0,김재철,18.0,1,NaN,NaN
6614,3478501093,9999,20200731.0,3.0,20200601.0,(주),탑인터큐브,NaN,"Top Inte Cube Co., Ltd.",M,2.0,2.0,1.0,2.0,N,N,NaN,10.0,46109.0,NaN,NaN,0.0,N,2987547.0,2019,1.0,NaN,"수출입업및동대행업, 무역관련 외 도매 및 소매업 외",NaN,0,이운용,30.0,1,NaN,NaN


In [70]:
휴폐업예측_train.columns

Index(['BIZ_NO', 'END_DATE', 'STRT_DATE', 'CLSBZ_GB', 'STAT_OCR_DATE',
       'CMP_PFIX_NM', 'CMP_NM', 'CMP_SFIX_NM', 'CMP_ENM', 'BZ_TYP', 'CMP_SCL',
       'PBCO_GB', 'PSN_CORP_GB', 'HDOF_BR_GB', 'FR_IVST_CORP_YN', 'VENT_YN',
       'LIST_CD', 'IND_CD_ORDR', 'IND_CD1', 'IND_CD2', 'IND_CD3', 'PB_ORG_TYP',
       'MDSCO_PRTC_YN', 'HDOF_CMP_CD', 'ESTB_DATE', 'ESTB_GB', 'LIST_DATE',
       'MN_BIZ_CONT', 'NATN_NM', 'HOMEPAGE_URL', 'CEO_NM', 'EMP_CNT', '부도',
       'CMP_NM1', 'ESTB_DATE_x'],
      dtype='object')

In [71]:
# 모델학습에 쓰지 않을 데이터들 날리기

휴폐업예측_train.drop(['CMP_PFIX_NM', 'CMP_SFIX_NM' , 'CMP_ENM' , 'CMP_SCL', 'PBCO_GB', 'PSN_CORP_GB', 'FR_IVST_CORP_YN','VENT_YN','LIST_CD','IND_CD_ORDR','IND_CD2','IND_CD3','PB_ORG_TYP','ESTB_GB','MN_BIZ_CONT','STRT_DATE'], axis = 1 , inplace = True)
휴폐업예측_test.drop(['CMP_PFIX_NM', 'CMP_SFIX_NM' , 'CMP_ENM' , 'CMP_SCL', 'PBCO_GB', 'PSN_CORP_GB', 'FR_IVST_CORP_YN','VENT_YN','LIST_CD','IND_CD_ORDR','IND_CD2','IND_CD3','PB_ORG_TYP','ESTB_GB','MN_BIZ_CONT','STRT_DATE'], axis = 1 , inplace = True)


In [72]:
휴폐업예측_train.head()

,BIZ_NO,END_DATE,CLSBZ_GB,STAT_OCR_DATE,CMP_NM,BZ_TYP,HDOF_BR_GB,IND_CD1,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,LIST_DATE,NATN_NM,HOMEPAGE_URL,CEO_NM,EMP_CNT,부도,CMP_NM1,ESTB_DATE_x
6212,4488700862,9999,3.0,20210923.0,유준,M,1.0,46499.0,N,NaN,2018,NaN,NaN,0,우대원,0.0,1,NaN,NaN
4233,1428700101,9999,3.0,20210930.0,세상,M,1.0,46311.0,N,NaN,2015,NaN,NaN,0,성광자,4.0,1,NaN,NaN
1055,4028158497,9999,3.0,20200925.0,고려상조,M,1.0,96921.0,N,NaN,2004,NaN,NaN,1,황병태,5.0,1,NaN,NaN
375,6088123771,9999,3.0,20220531.0,삼영설비,M,1.0,42201.0,N,NaN,1995,NaN,NaN,0,김재철,18.0,1,NaN,NaN
6614,3478501093,9999,3.0,20200601.0,탑인터큐브,M,2.0,46109.0,N,2987547.0,2019,NaN,NaN,0,이운용,30.0,1,NaN,NaN


In [73]:
# IND_CD1 산업코드 앞의 두자릿수만 자르기 
휴폐업예측_train['IND_CD1'] = 휴폐업예측_train['IND_CD1'].astype('str').str[:2]
휴폐업예측_test['IND_CD1'] = 휴폐업예측_test['IND_CD1'].astype('str').str[:2]


In [74]:
# IND_CD1  na는 0으로 일단 임의로 바꿈 
휴폐업예측_train['IND_CD1'] = 휴폐업예측_train['IND_CD1'].replace('na', 0)
휴폐업예측_test['IND_CD1'] = 휴폐업예측_test['IND_CD1'].replace('na', 0)


# 중견기업보호여부 
* Y > 1 , N > 0

In [75]:
휴폐업예측_train['MDSCO_PRTC_YN'].value_counts()

N    28292
Y       83
Name: MDSCO_PRTC_YN, dtype: int64

In [76]:
# df_test['Title'] = df_test['Title'].replace(['Mlle','Ms','Mme'], 'Miss')  중견기업보호여부 Y > 1 / N > 0
휴폐업예측_train['MDSCO_PRTC_YN'] = 휴폐업예측_train['MDSCO_PRTC_YN'].replace('N', 0)
휴폐업예측_train['MDSCO_PRTC_YN'] = 휴폐업예측_train['MDSCO_PRTC_YN'].replace('Y', 1)

휴폐업예측_test['MDSCO_PRTC_YN'] = 휴폐업예측_test['MDSCO_PRTC_YN'].replace('N', 0)
휴폐업예측_test['MDSCO_PRTC_YN'] = 휴폐업예측_test['MDSCO_PRTC_YN'].replace('Y', 1)

In [77]:
휴폐업예측_train['IND_CD1'] = 휴폐업예측_train['IND_CD1'].astype('int64')
휴폐업예측_test['IND_CD1'] = 휴폐업예측_test['IND_CD1'].astype('int64')

In [78]:
휴폐업예측_train.head()

,BIZ_NO,END_DATE,CLSBZ_GB,STAT_OCR_DATE,CMP_NM,BZ_TYP,HDOF_BR_GB,IND_CD1,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,LIST_DATE,NATN_NM,HOMEPAGE_URL,CEO_NM,EMP_CNT,부도,CMP_NM1,ESTB_DATE_x
6212,4488700862,9999,3.0,20210923.0,유준,M,1.0,46,0.0,NaN,2018,NaN,NaN,0,우대원,0.0,1,NaN,NaN
4233,1428700101,9999,3.0,20210930.0,세상,M,1.0,46,0.0,NaN,2015,NaN,NaN,0,성광자,4.0,1,NaN,NaN
1055,4028158497,9999,3.0,20200925.0,고려상조,M,1.0,96,0.0,NaN,2004,NaN,NaN,1,황병태,5.0,1,NaN,NaN
375,6088123771,9999,3.0,20220531.0,삼영설비,M,1.0,42,0.0,NaN,1995,NaN,NaN,0,김재철,18.0,1,NaN,NaN
6614,3478501093,9999,3.0,20200601.0,탑인터큐브,M,2.0,46,0.0,2987547.0,2019,NaN,NaN,0,이운용,30.0,1,NaN,NaN


In [79]:
# 1 아니면 0으로 다 바꿔버리기 
def change0not1(data):
    # if data != "":
    if data != 1:
        data = 0
        return data
    else:
        return data

# 대표수
* 대표자 1인 > 0 , 대표자 2인 이상 > 1 


In [80]:
# 대표 1인 0 ,, 대표 2인 이상이면 1 
# 휴폐업예측_train['CEO_NM'] = np.where(휴폐업예측_train['CEO_NM'].str.contains(",") , 0 , 휴폐업예측_train['CEO_NM'])
휴폐업예측_train['CEO_NM'] = np.where(휴폐업예측_train['CEO_NM'].str.contains("외") , 1 , 휴폐업예측_train['CEO_NM']) # 공동대표인 애들은 1로 
휴폐업예측_train['CEO_NM'] = 휴폐업예측_train['CEO_NM'].apply(change0not1)  # 1아닌 나머지 애들은 함수 적용해서 0으로

휴폐업예측_test['CEO_NM'] = np.where(휴폐업예측_test['CEO_NM'].str.contains("외") , 1 , 휴폐업예측_test['CEO_NM'])
휴폐업예측_test['CEO_NM'] = 휴폐업예측_test['CEO_NM'].apply(change0not1)

In [81]:
휴폐업예측_train['CEO_NM'].value_counts()


1    23308
0     5072
Name: CEO_NM, dtype: int64

In [82]:
휴폐업예측_train['NATN_NM'].isna().sum()

28259

# 국가 
* 한국은 1, 국외는 0 

In [83]:
# NATN_NM 임의로 한국은 1 , 국외는 0으로 임의로 잡음 
휴폐업예측_train['NATN_NM'] = 휴폐업예측_train['NATN_NM'].fillna(1)
휴폐업예측_test['NATN_NM'] = 휴폐업예측_test['NATN_NM'].fillna(1)
# 1이 아닌 애들은 다 0으로 바꾸긔까지 클릐어잼잼이
휴폐업예측_train['NATN_NM'] = 휴폐업예측_train['NATN_NM'].apply(change0not1)
휴폐업예측_test['NATN_NM'] = 휴폐업예측_test['NATN_NM'].apply(change0not1)

In [84]:
휴폐업예측_train['NATN_NM'] = 휴폐업예측_train['NATN_NM'].astype('int64')
휴폐업예측_test['NATN_NM'] = 휴폐업예측_test['NATN_NM'].astype('int64')

In [85]:
휴폐업예측_train['NATN_NM'].value_counts()

1    28259
0      121
Name: NATN_NM, dtype: int64

In [86]:
휴폐업예측_train.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 28380 entries, 6212 to 27036
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   BIZ_NO         28380 non-null  int64  
 1   END_DATE       28380 non-null  object 
 2   CLSBZ_GB       5472 non-null   float64
 3   STAT_OCR_DATE  5472 non-null   float64
 4   CMP_NM         5467 non-null   object 
 5   BZ_TYP         28375 non-null  object 
 6   HDOF_BR_GB     28375 non-null  float64
 7   IND_CD1        28380 non-null  int64  
 8   MDSCO_PRTC_YN  28375 non-null  float64
 9   HDOF_CMP_CD    3521 non-null   float64
 10  ESTB_DATE      28380 non-null  object 
 11  LIST_DATE      40 non-null     float64
 12  NATN_NM        28380 non-null  int64  
 13  HOMEPAGE_URL   28380 non-null  int64  
 14  CEO_NM         28380 non-null  int64  
 15  EMP_CNT        4903 non-null   float64
 16  부도             28380 non-null  int64  
 17  CMP_NM1        22908 non-null  object 
 18  EST

In [87]:
휴폐업예측_train.head()

,BIZ_NO,END_DATE,CLSBZ_GB,STAT_OCR_DATE,CMP_NM,BZ_TYP,HDOF_BR_GB,IND_CD1,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,LIST_DATE,NATN_NM,HOMEPAGE_URL,CEO_NM,EMP_CNT,부도,CMP_NM1,ESTB_DATE_x
6212,4488700862,9999,3.0,20210923.0,유준,M,1.0,46,0.0,NaN,2018,NaN,1,0,0,0.0,1,NaN,NaN
4233,1428700101,9999,3.0,20210930.0,세상,M,1.0,46,0.0,NaN,2015,NaN,1,0,0,4.0,1,NaN,NaN
1055,4028158497,9999,3.0,20200925.0,고려상조,M,1.0,96,0.0,NaN,2004,NaN,1,1,0,5.0,1,NaN,NaN
375,6088123771,9999,3.0,20220531.0,삼영설비,M,1.0,42,0.0,NaN,1995,NaN,1,0,0,18.0,1,NaN,NaN
6614,3478501093,9999,3.0,20200601.0,탑인터큐브,M,2.0,46,0.0,2987547.0,2019,NaN,1,0,0,30.0,1,NaN,NaN


# 산업코드(BZ_TYP)

In [88]:
휴폐업예측_train.BZ_TYP.value_counts()

M    28054
O      240
A       40
V       32
S        6
F        1
I        1
B        1
Name: BZ_TYP, dtype: int64

In [89]:
# bz type하고 산업코드 col을 하나로 만들어야겠다고 생각 

In [90]:
휴폐업예측_train.IND_CD1.nunique() # chd 69개로 구분되어 있음

69

In [91]:
휴폐업예측_train['IND_CD1_2'] = 휴폐업예측_train['IND_CD1'].astype('str').str[:1] # 군집 수 줄이기 위해 첫글자만 이용 


In [92]:
휴폐업예측_train.IND_CD1_2.nunique()

10

In [93]:
휴폐업예측_train.IND_CD1_2.value_counts()

4    7580
2    6954
6    4513
1    2234
3    2139
7    2020
5    1977
9     625
8     208
0     130
Name: IND_CD1_2, dtype: int64

In [94]:
pd.crosstab(휴폐업예측_train.IND_CD1_2,휴폐업예측_train.BZ_TYP) 
#A신금(신용금고) B은행 C카드 F생보 I손보 M 제조 O기타금융 S증권 V 창투(창업투자회사) 

BZ_TYP,A,B,F,I,M,O,S,V
IND_CD1_2,,,,,,,,
0,0,0,0,0,125,0,0,0
1,0,0,0,0,2234,0,0,0
2,0,0,0,0,6953,1,0,0
3,0,0,0,0,2139,0,0,0
4,0,0,0,0,7575,5,0,0
5,0,0,0,0,1976,1,0,0
6,40,0,1,1,4216,221,6,28
7,0,1,0,0,2003,12,0,4
8,0,0,0,0,208,0,0,0


In [95]:
idx_list = 휴폐업예측_train[(휴폐업예측_train['IND_CD1_2']=='6') & (휴폐업예측_train['BZ_TYP'] != 'M')].index

휴폐업예측_train.loc[idx_list, 'IND_CD1_2'] = '10'

In [96]:
휴폐업예측_train['IND_CD1_2'].value_counts()

4     7563
2     6933
6     4213
1     2227
3     2136
7     2017
5     1970
9      624
10     359
8      208
0      130
Name: IND_CD1_2, dtype: int64

In [97]:
# 산업코드 0~9 모두 제조업 섞여있다
# 산업코드 6번만 더 세분화 해주면 될거같다  제조 외에 10번 코드로 금융  

# 본점, 지점 칼럼 쓸 수 있는지 확인 > 실패

In [98]:
휴폐업예측_train.HDOF_BR_GB.value_counts() #1이 본점

1.0    24839
2.0     3536
Name: HDOF_BR_GB, dtype: int64

In [99]:
휴폐업예측_train.HDOF_CMP_CD.value_counts()

1308524.0    171
1857086.0     35
1365262.0     35
1493057.0     28
1713224.0     23
            ... 
1436241.0      1
1973167.0      1
1548222.0      1
1780029.0      1
3848307.0      1
Name: HDOF_CMP_CD, Length: 2109, dtype: int64

In [100]:
df_HDOF=휴폐업예측_train.HDOF_CMP_CD.value_counts()#지점의 본점코드를 조회해서 중복 확인 

In [101]:
df_HDOF=df_HDOF.reset_index()

In [102]:
df_HDOF['BIZ_NO']=df_HDOF['index']
del df_HDOF['index']
df_HDOF

,HDOF_CMP_CD,BIZ_NO
0,171,1308524.0
1,35,1857086.0
2,35,1365262.0
3,28,1493057.0
4,23,1713224.0
...,...,...
2104,1,1436241.0
2105,1,1973167.0
2106,1,1548222.0
2107,1,1780029.0


In [103]:
# 중복되는 데이터 발견되는데, 본점 코드가 뭔지 몰라서 활용 불가 >> 실패 

In [104]:
# 상태발생일자도 4자리만 떼어줌 혹시 몰라서 연월일 다 붙어있는 것도 놔둠 
휴폐업예측_train['STAT_OCR_DATE4']=  휴폐업예측_train['STAT_OCR_DATE'].astype('str').str[:4]
휴폐업예측_test['STAT_OCR_DATE4']=  휴폐업예측_test['STAT_OCR_DATE'].astype('str').str[:4]


In [105]:
휴폐업예측_train.head()

,BIZ_NO,END_DATE,CLSBZ_GB,STAT_OCR_DATE,CMP_NM,BZ_TYP,HDOF_BR_GB,IND_CD1,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,LIST_DATE,NATN_NM,HOMEPAGE_URL,CEO_NM,EMP_CNT,부도,CMP_NM1,ESTB_DATE_x,IND_CD1_2,STAT_OCR_DATE4
6212,4488700862,9999,3.0,20210923.0,유준,M,1.0,46,0.0,NaN,2018,NaN,1,0,0,0.0,1,NaN,NaN,4,2021
4233,1428700101,9999,3.0,20210930.0,세상,M,1.0,46,0.0,NaN,2015,NaN,1,0,0,4.0,1,NaN,NaN,4,2021
1055,4028158497,9999,3.0,20200925.0,고려상조,M,1.0,96,0.0,NaN,2004,NaN,1,1,0,5.0,1,NaN,NaN,9,2020
375,6088123771,9999,3.0,20220531.0,삼영설비,M,1.0,42,0.0,NaN,1995,NaN,1,0,0,18.0,1,NaN,NaN,4,2022
6614,3478501093,9999,3.0,20200601.0,탑인터큐브,M,2.0,46,0.0,2987547.0,2019,NaN,1,0,0,30.0,1,NaN,NaN,4,2020


# 부도상태 지수

In [106]:
# 부도상태 지수 파생변수 붙이기 
휴폐업예측_train[['END_DATE','STAT_OCR_DATE4']]

,END_DATE,STAT_OCR_DATE4
6212,9999,2021
4233,9999,2021
1055,9999,2020
375,9999,2022
6614,9999,2020
...,...,...
28044,nan,nan
11601,nan,nan
2789,nan,nan
16695,nan,nan


In [107]:
휴폐업예측_train['END_DATE'].value_counts()
# 휴폐업예측_test['END_DATE'].value_counts()


nan     22908
9999     5393
2020       33
2019       20
2021       17
2022        7
2018        2
Name: END_DATE, dtype: int64

In [108]:
휴폐업예측_train['ESTB_DATE'].value_counts() 
휴폐업예측_train['STAT_OCR_DATE4'].value_counts() 

nan     22908
2021     1965
2020     1691
2022      984
2019      733
2018       99
Name: STAT_OCR_DATE4, dtype: int64

In [109]:
휴폐업예측_train["END_DATE"] = 휴폐업예측_train["END_DATE"].replace( "nan", "9999") # nan값은 액티브에서 딸려온 것이기 때문에 최근까지 운영중임 9999로 바꿔줌! 
# 아 근데 생각해보니 ^^.. 액티브에서도 휴폐업 했던 애들이 있었던 것 같은데 ^^ 일단 순액티브만 쓰기로 했으니까 상관없음..ㅎㅎ

# nan값은 액티브에서 딸려온 것이기 때문에 최근까지 운영중임 9999로 바꿔줌! 
휴폐업예측_test["END_DATE"] = 휴폐업예측_test["END_DATE"].replace( "nan", "9999") 

휴폐업예측_train["STAT_OCR_DATE4"] = 휴폐업예측_train["STAT_OCR_DATE4"].replace( "nan", 0) 
휴폐업예측_test["STAT_OCR_DATE4"] = 휴폐업예측_test["STAT_OCR_DATE4"].replace( "nan", 0) 


In [110]:

# 이건 임의로 nan값 2015로 지정해준거니까 나중에 지워버릴 것 !!! 
휴폐업예측_train["ESTB_DATE"] = 휴폐업예측_train["ESTB_DATE"].replace("nan", "9999") 
휴폐업예측_test["ESTB_DATE"] = 휴폐업예측_test["ESTB_DATE"].replace("nan", "9999") 


In [111]:
# str에서 int형으로 바꿔줌!
휴폐업예측_train['END_DATE'] = 휴폐업예측_train['END_DATE'].astype('int64')  
휴폐업예측_train['ESTB_DATE'] = 휴폐업예측_train['ESTB_DATE'].astype('int64') 
휴폐업예측_train['STAT_OCR_DATE4'] = 휴폐업예측_train['STAT_OCR_DATE4'].astype('int64') 


In [112]:
휴폐업예측_train['END_DATE'].value_counts()

9999    28301
2020       33
2019       20
2021       17
2022        7
2018        2
Name: END_DATE, dtype: int64

In [113]:
휴폐업예측_train['STAT_OCR_DATE4'].value_counts()

0       22908
2021     1965
2020     1691
2022      984
2019      733
2018       99
Name: STAT_OCR_DATE4, dtype: int64

In [ ]:
휴폐업예측_train['ESTB_DATE'].value_counts()

# 2018 1082 / 2019 836 / 2020 596 / 2021 56 / 2022 2 / 9999 1316
# 836 + 596 + 56 + 2 

In [115]:
(휴폐업예측_train['END_DATE'] - 휴폐업예측_train['STAT_OCR_DATE4']).value_counts()

# 9999 >> 순액티브애들 9999-0
# 7978 >> 9999 - 7978 = 2021 년에 휴폐업 상태~ 최근까지
# 7979 >> 9999 - 7979 = 2020 년에 휴폐업 상태~ 최근까지 
# 7977 >> 9999 - 7977 = 2022 년에 휴폐업 상태~ 최근까지 
# 7980 >> 9999 - 7980 = 2019 년에 휴폐업 상태~ 최근까지
# 7981 >> 9999 - 7981 = 2018 년에 휴폐업 상태~ 최근까지
# 0    >> 같은 년도에 휴폐업이 발생했다가 끝난 애들 
# 1    >> ex. 2018~2019 연속 2년간 휴폐업인 애들  
# 2    >> ex. 2018~2020 연속 3년간 휴폐업인 애들 

9999    22908
7978     1952
7979     1656
7977      982
7980      710
7981       93
0          56
1          21
2           2
dtype: int64

In [116]:
9999-7981 

2018

In [117]:
휴폐업예측_train['휴폐업예측위한변수'] = 휴폐업예측_train['END_DATE'] - 휴폐업예측_train['STAT_OCR_DATE4']

In [118]:
# 자리만들어주기 
휴폐업예측_train['2018_부도'] = 0 
휴폐업예측_train['2019_부도'] = 0
휴폐업예측_train['2020_부도'] = 0
휴폐업예측_train['2021_부도'] = 0
휴폐업예측_train['2022_부도'] = 0

In [119]:
# 설립일 이후인 애들부터 표시해주자 일단 해당연도에 설립안된 애들은 2로 채워줄거고 얘넨 다 액티브이기 때문에 2가 아닌 다른건 다 0으로 채워넣어주면 됨 
# for i in range(len(휴폐업예측_train)):
#   if 휴폐업예측_train["ESTB_DATE"][i] == 2018:
#     휴폐업예측_train['2018_부도'][i] = 0

# 2018년 이후에 설립된 기업들 
idx_list = 휴폐업예측_train[(휴폐업예측_train['ESTB_DATE']==2018)].index
휴폐업예측_train.loc[idx_list, '2018_부도'] = 0
휴폐업예측_train.loc[idx_list, '2019_부도'] = 0
휴폐업예측_train.loc[idx_list, '2020_부도'] = 0
휴폐업예측_train.loc[idx_list, '2021_부도'] = 0
휴폐업예측_train.loc[idx_list, '2022_부도'] = 0


In [120]:
# # 18년 이전에 설립된 기업들 (9999) 
# idx_list_b18 = 휴폐업예측_train[(휴폐업예측_train['ESTB_DATE']==9999)].index
# 휴폐업예측_train.loc[idx_list_b18, '2018_부도'] = 0
# 휴폐업예측_train.loc[idx_list_b18, '2019_부도'] = 0
# 휴폐업예측_train.loc[idx_list_b18, '2020_부도'] = 0
# 휴폐업예측_train.loc[idx_list_b18, '2021_부도'] = 0
# 휴폐업예측_train.loc[idx_list_b18, '2022_부도'] = 0

In [121]:
# 2019년 이후에 설립된 기업들 
idx_list_19 = 휴폐업예측_train[(휴폐업예측_train['ESTB_DATE']==2019)].index
휴폐업예측_train.loc[idx_list_19, '2018_부도'] = 2
휴폐업예측_train.loc[idx_list_19, '2019_부도'] = 0
휴폐업예측_train.loc[idx_list_19, '2020_부도'] = 0
휴폐업예측_train.loc[idx_list_19, '2021_부도'] = 0
휴폐업예측_train.loc[idx_list_19, '2022_부도'] = 0

In [122]:
# 2020년 이후에 설립된 기업들 
idx_list_20 = 휴폐업예측_train[(휴폐업예측_train['ESTB_DATE']==2020)].index
휴폐업예측_train.loc[idx_list_20, '2018_부도'] = 2
휴폐업예측_train.loc[idx_list_20, '2019_부도'] = 2
휴폐업예측_train.loc[idx_list_20, '2020_부도'] = 0
휴폐업예측_train.loc[idx_list_20, '2021_부도'] = 0
휴폐업예측_train.loc[idx_list_20, '2022_부도'] = 0

In [123]:
# 2021년 이후에 설립된 기업들 
idx_list_21 = 휴폐업예측_train[(휴폐업예측_train['ESTB_DATE']==2021)].index
휴폐업예측_train.loc[idx_list_21, '2018_부도'] = 2
휴폐업예측_train.loc[idx_list_21, '2019_부도'] = 2
휴폐업예측_train.loc[idx_list_21, '2020_부도'] = 2
휴폐업예측_train.loc[idx_list_21, '2021_부도'] = 0
휴폐업예측_train.loc[idx_list_21, '2022_부도'] = 0

In [124]:
# 2022년 이후에 설립된 기업들 
idx_list_22 = 휴폐업예측_train[(휴폐업예측_train['ESTB_DATE']==2022)].index
휴폐업예측_train.loc[idx_list_22, '2018_부도'] = 2
휴폐업예측_train.loc[idx_list_22, '2019_부도'] = 2
휴폐업예측_train.loc[idx_list_22, '2020_부도'] = 2
휴폐업예측_train.loc[idx_list_22, '2021_부도'] = 2
휴폐업예측_train.loc[idx_list_22, '2022_부도'] = 0

In [125]:
휴폐업예측_train["2018_부도"].value_counts()

# 19년도까지 채워넣어줬을 때 (0    27411   / 2      957)
# 20년도까지 채워넣어줬을 때 (0   26780   / 2       1588)
# 21년도까지 채워넣어줬을 때 (0   26723   / 2       1645)
# 22년도까지 채워넣어줬을 때 (0   26721   / 2       1647)


0    28017
2      363
Name: 2018_부도, dtype: int64

In [126]:
# (휴폐업예측_train['END_DATE'] - 휴폐업예측_train['STAT_OCR_DATE4']).value_counts()

# 9999 >> 순액티브애들 9999-0
# 7978 >> 9999 - 7978 = 2021 년에 휴폐업 상태~ 최근까지
# 7979 >> 9999 - 7979 = 2020 년에 휴폐업 상태~ 최근까지 
# 7977 >> 9999 - 7977 = 2022 년에 휴폐업 상태~ 최근까지 
# 7980 >> 9999 - 7980 = 2019 년에 휴폐업 상태~ 최근까지
# 7981 >> 9999 - 7981 = 2018 년에 휴폐업 상태~ 최근까지
# 0    >> 같은 년도에 휴폐업이 발생했다가 끝난 애들 
# 1    >> ex. 2018~2019 연속 2년간 휴폐업인 애들  
# 2    >> ex. 2018~2020 연속 3년간 휴폐업인 애들 

In [127]:
휴폐업예측_train['예측을위한'] = 휴폐업예측_train['END_DATE'] - 휴폐업예측_train['STAT_OCR_DATE4']

In [128]:
# enddate - stat_ocr_date 기준으로 부도상태 파생변수 채워넣어주기 (핵심잼)

# 2021년 ~ 지금까지 폐업인 애들 (7978)
idx_list_c21 = 휴폐업예측_train[(휴폐업예측_train['예측을위한']==7978)].index
휴폐업예측_train.loc[idx_list_c21, '2018_부도'] = 0
휴폐업예측_train.loc[idx_list_c21, '2019_부도'] = 0
휴폐업예측_train.loc[idx_list_c21, '2020_부도'] = 0
휴폐업예측_train.loc[idx_list_c21, '2021_부도'] = 1
휴폐업예측_train.loc[idx_list_c21, '2022_부도'] = 1

# 2020년 ~ 지금까지 폐업인 애들 (7979)
idx_list_c20 = 휴폐업예측_train[(휴폐업예측_train['예측을위한']==7979)].index
휴폐업예측_train.loc[idx_list_c20, '2018_부도'] = 0
휴폐업예측_train.loc[idx_list_c20, '2019_부도'] = 0
휴폐업예측_train.loc[idx_list_c20, '2020_부도'] = 1
휴폐업예측_train.loc[idx_list_c20, '2021_부도'] = 1
휴폐업예측_train.loc[idx_list_c20, '2022_부도'] = 1

# 2022년 ~ 지금까지 폐업인 애들 (7977)
idx_list_c20 = 휴폐업예측_train[(휴폐업예측_train['예측을위한']==7979)].index
휴폐업예측_train.loc[idx_list_c20, '2018_부도'] = 0
휴폐업예측_train.loc[idx_list_c20, '2019_부도'] = 0
휴폐업예측_train.loc[idx_list_c20, '2020_부도'] = 0
휴폐업예측_train.loc[idx_list_c20, '2021_부도'] = 0
휴폐업예측_train.loc[idx_list_c20, '2022_부도'] = 1

# 2019년 ~ 지금까지 폐업인 애들 (7980)
idx_list_c19 = 휴폐업예측_train[(휴폐업예측_train['예측을위한']==7980)].index
휴폐업예측_train.loc[idx_list_c19, '2018_부도'] = 0
휴폐업예측_train.loc[idx_list_c19, '2019_부도'] = 1
휴폐업예측_train.loc[idx_list_c19, '2020_부도'] = 1
휴폐업예측_train.loc[idx_list_c19, '2021_부도'] = 1
휴폐업예측_train.loc[idx_list_c19, '2022_부도'] = 1

# 2018년 ~ 지금까지 폐업인 애들 (7981)
idx_list_c18 = 휴폐업예측_train[(휴폐업예측_train['예측을위한']==7981)].index
휴폐업예측_train.loc[idx_list_c18, '2018_부도'] = 1
휴폐업예측_train.loc[idx_list_c18, '2019_부도'] = 1
휴폐업예측_train.loc[idx_list_c18, '2020_부도'] = 1
휴폐업예측_train.loc[idx_list_c18, '2021_부도'] = 1
휴폐업예측_train.loc[idx_list_c18, '2022_부도'] = 1

In [129]:
#  휴폐업예측_train['END_DATE'] - 휴폐업예측_train['STAT_OCR_DATE4'] 0,1,2 인 애들만 따로 뽑아보기 
a = 휴폐업예측_train[(휴폐업예측_train["예측을위한"] <= 2)]
len(a)

79

# 18~20년 사이에 한번이라도 휴폐업 상태발생한 기업들의 부도상태지수
* 영잼, 일잼, 이잼

In [130]:
# 영잼, 일잼 , 이잼 
영잼 = 휴폐업예측_train[(휴폐업예측_train["예측을위한"] == 0)][['STAT_OCR_DATE4','END_DATE','2018_부도','2019_부도','2020_부도','2021_부도','2022_부도']]
len(영잼)
영잼['STAT_OCR_DATE4'].value_counts()

2020    27
2019    17
2021     8
2018     2
2022     2
Name: STAT_OCR_DATE4, dtype: int64

In [131]:
# 현재까지 
휴폐업예측_train["2018_부도"].value_counts()  # 1 >> 165
휴폐업예측_train["2019_부도"].value_counts()  # 1 >> 1442
휴폐업예측_train["2020_부도"].value_counts()  # 1 >> 1442
휴폐업예측_train["2021_부도"].value_counts()  # 1 >> 4935
휴폐업예측_train["2022_부도"].value_counts()  # 1 >> 7887


0    20421
1     7959
Name: 2022_부도, dtype: int64

In [132]:
# 0인 애들은 해당연도에 휴폐업 상태가 다 끝난 애들 
idx_list_영잼_18 = 휴폐업예측_train[(휴폐업예측_train['예측을위한']==0) & (휴폐업예측_train['STAT_OCR_DATE4'] == 2018) ].index
휴폐업예측_train.loc[idx_list_영잼_18, '2018_부도'] = 1

idx_list_영잼_19 = 휴폐업예측_train[(휴폐업예측_train['예측을위한']==0) & (휴폐업예측_train['STAT_OCR_DATE4'] == 2019 )].index
휴폐업예측_train.loc[idx_list_영잼_19, '2019_부도'] = 1

idx_list_영잼_20 = 휴폐업예측_train[(휴폐업예측_train['예측을위한']==0) & (휴폐업예측_train['STAT_OCR_DATE4'] == 2020 )].index
휴폐업예측_train.loc[idx_list_영잼_20, '2020_부도'] = 1

idx_list_영잼_21 = 휴폐업예측_train[(휴폐업예측_train['예측을위한']==0) & (휴폐업예측_train['STAT_OCR_DATE4'] == 2021 )].index
휴폐업예측_train.loc[idx_list_영잼_21, '2021_부도'] = 1

idx_list_영잼_22 = 휴폐업예측_train[(휴폐업예측_train['예측을위한']==0) & (휴폐업예측_train['STAT_OCR_DATE4'] == 2022) ].index
휴폐업예측_train.loc[idx_list_영잼_22, '2022_부도'] = 1

In [133]:
# 현재까지 
휴폐업예측_train["2018_부도"].value_counts()  # 1 >> 165
휴폐업예측_train["2019_부도"].value_counts()  # 1 >> 1442
휴폐업예측_train["2020_부도"].value_counts()  # 1 >> 1442
휴폐업예측_train["2021_부도"].value_counts()  # 1 >> 4935
휴폐업예측_train["2022_부도"].value_counts()  # 1 >> 7887


0    20418
1     7962
Name: 2022_부도, dtype: int64

In [134]:
# 0, 1, 2 인 기업들만 따로 떼어내서 만들어주기
영잼 = 휴폐업예측_train[(휴폐업예측_train["예측을위한"] == 0)][['STAT_OCR_DATE4','END_DATE','2018_부도','2019_부도','2020_부도','2021_부도','2022_부도']]
영잼

,STAT_OCR_DATE4,END_DATE,2018_부도,2019_부도,2020_부도,2021_부도,2022_부도
2803,2020,2020,0,0,1,0,0
2299,2020,2020,0,0,1,0,0
4742,2020,2020,0,0,1,0,0
388,2019,2019,0,1,0,0,0
675,2020,2020,0,0,1,0,0
3893,2020,2020,0,0,1,0,0
4322,2020,2020,0,0,1,0,0
5570,2020,2020,0,0,1,0,0
556,2020,2020,0,0,1,0,0
4668,2018,2018,1,0,0,0,0


In [135]:
 # 일잼 / 2018~ 2019 이런식으로 연속 2년간 휴폐업인 경우임 
일잼 = 휴폐업예측_train[(휴폐업예측_train["예측을위한"] == 1)][['STAT_OCR_DATE4','END_DATE','2018_부도','2019_부도','2020_부도','2021_부도','2022_부도']]
일잼 # 21개

,STAT_OCR_DATE4,END_DATE,2018_부도,2019_부도,2020_부도,2021_부도,2022_부도
1871,2020,2021,0,0,0,0,0
6117,2021,2022,0,0,0,0,0
1908,2019,2020,0,0,0,0,0
5422,2020,2021,0,0,0,0,0
509,2019,2020,0,0,0,0,0
3056,2020,2021,0,0,0,0,0
928,2021,2022,0,0,0,0,0
1888,2021,2022,0,0,0,0,0
2615,2020,2021,0,0,0,0,0
2545,2019,2020,0,0,0,0,0


In [136]:
일잼['STAT_OCR_DATE4'].value_counts()

2020    8
2021    5
2019    5
2018    3
Name: STAT_OCR_DATE4, dtype: int64

In [137]:
idx_list_일잼_18 = 휴폐업예측_train[(휴폐업예측_train['예측을위한']==1) & (휴폐업예측_train['STAT_OCR_DATE4'] == 2018) ].index
휴폐업예측_train.loc[idx_list_일잼_18, ['2018_부도','2019_부도']] = 1

idx_list_일잼_19 = 휴폐업예측_train[(휴폐업예측_train['예측을위한']==1) & (휴폐업예측_train['STAT_OCR_DATE4'] == 2019) ].index
휴폐업예측_train.loc[idx_list_일잼_19, ['2019_부도','2020_부도']] = 1

idx_list_일잼_20 = 휴폐업예측_train[(휴폐업예측_train['예측을위한']==1) & (휴폐업예측_train['STAT_OCR_DATE4'] == 2020) ].index
휴폐업예측_train.loc[idx_list_일잼_20, ['2020_부도','2021_부도']] = 1

idx_list_일잼_21 = 휴폐업예측_train[(휴폐업예측_train['예측을위한']==1) & (휴폐업예측_train['STAT_OCR_DATE4'] == 2021) ].index
휴폐업예측_train.loc[idx_list_일잼_21, ['2021_부도','2022_부도']] = 1

In [138]:
# 확인
일잼 = 휴폐업예측_train[(휴폐업예측_train["예측을위한"] == 1)][['STAT_OCR_DATE4','END_DATE','2018_부도','2019_부도','2020_부도','2021_부도','2022_부도']]
일잼 # 21개

,STAT_OCR_DATE4,END_DATE,2018_부도,2019_부도,2020_부도,2021_부도,2022_부도
1871,2020,2021,0,0,1,1,0
6117,2021,2022,0,0,0,1,1
1908,2019,2020,0,1,1,0,0
5422,2020,2021,0,0,1,1,0
509,2019,2020,0,1,1,0,0
3056,2020,2021,0,0,1,1,0
928,2021,2022,0,0,0,1,1
1888,2021,2022,0,0,0,1,1
2615,2020,2021,0,0,1,1,0
2545,2019,2020,0,1,1,0,0


In [139]:
이잼 = 휴폐업예측_train[(휴폐업예측_train["예측을위한"] == 2)][['STAT_OCR_DATE4','END_DATE','2018_부도','2019_부도','2020_부도','2021_부도','2022_부도']]
이잼 

,STAT_OCR_DATE4,END_DATE,2018_부도,2019_부도,2020_부도,2021_부도,2022_부도
4331,2019,2021,0,0,0,0,0
2964,2018,2020,0,0,0,0,0


In [140]:
# 이잼 
idx_list_일잼_18 = 휴폐업예측_train[(휴폐업예측_train['예측을위한']==2) & (휴폐업예측_train['STAT_OCR_DATE4'] == 2018) ].index
휴폐업예측_train.loc[idx_list_일잼_18, ['2018_부도','2019_부도','2020_부도']] = 1

idx_list_일잼_19 = 휴폐업예측_train[(휴폐업예측_train['예측을위한']==2) & (휴폐업예측_train['STAT_OCR_DATE4'] == 2019) ].index
휴폐업예측_train.loc[idx_list_일잼_19, ['2019_부도','2020_부도','2021_부도']] = 1

In [141]:
# 확인
이잼 = 휴폐업예측_train[(휴폐업예측_train["예측을위한"] == 2)][['STAT_OCR_DATE4','END_DATE','2018_부도','2019_부도','2020_부도','2021_부도','2022_부도']]
이잼 

,STAT_OCR_DATE4,END_DATE,2018_부도,2019_부도,2020_부도,2021_부도,2022_부도
4331,2019,2021,0,1,1,1,0
2964,2018,2020,1,1,1,0,0


In [142]:
휴폐업예측_train['2022_부도'].value_counts()


# 최종 
# 2018부도상태  (0    26724 // 1     1470 // 2      174 )
# 2019부도상태  (0    26210 // 1    1493 // 2      665 )
# 2020부도상태  (0    26793 // 1     1516 // 2      59 )
# 2021부도상태  (0    23389 // 1     4977 // 2      2 )
# 2022부도상태  (0    20467 // 1     7901 // 2      0 )


0    20408
1     7972
Name: 2022_부도, dtype: int64

In [143]:
# 휴폐업예측_train.to_excel("./휴폐업예측_train.xlsx", index=False)
# 휴폐업예측_test.to_excel("./휴폐업예측_test.xlsx", index=False)


# 크롤링

In [144]:
import sqlalchemy
from sqlalchemy.types import Integer, String, DateTime
from bs4 import BeautifulSoup
import requests

In [145]:
# res = requests.get("https://www.incruit.com/company/4665501")
# htmlstr = res.text
# soup = BeautifulSoup(htmlstr,features="html.parser")
# li_list = soup.select_one("#company_warp > div:nth-child(4) > div > div:nth-child(6) > div > ul > li:nth-child(1) ").text

In [146]:
# res = requests.get("https://www.nicebizinfo.com/ep/EP0100M002GE.nice?kiscode=030589")
# htmlstr = res.text
# soup = BeautifulSoup(htmlstr,features="html.parser")
# li_list = soup(" body > div:nth-child(31) > div > div.cTable.sp2.mb10 > table > tbody > tr:nth-child(1) > td:nth-child(2) > div ")

# 워드클라우드(크롤링)

In [147]:
def ytn_news():
    ---------------------------- 오래 걸려서 막음 --------------------------
    res = requests.get("https://www.ytn.co.kr/news/list.php?mcd=S0018")
    soup = BeautifulSoup(res.text)
    news_list = soup.select('div#nav_content > div:nth-child(1) > ul > li')
    titles = []
    for news in news_list:
        title = news.select_one("a > div.infowrap > span").text
        title = re.sub(r"[/w]", "", title)  # 문자+숫자(alphanumeric)와 매치	[a-zA-Z0-9ㄱ-ㅎㅏ-ㅣ_$%]
        title = re.sub(r'[\"\'\[\]\,\.\\]', " ", title)  # 문자+숫자(alphanumeric)와 매치	[a-zA-Z0-9ㄱ-ㅎㅏ-ㅣ_$%]
        titles.append(title)  # .strip().replace('\n',' '))
        # titles.append(title.strip().replace('\n',' ')
        #                           .replace("\\'",'')
        #                           .replace("'",'')
        #                           .replace('"',''))
    print(titles)
    titles = ['50대 4차 접종 시작    휴일 검사기관 확대 필요 ', '피서지 코로나 확진자 급증   지자체 방역  비상 ', '코로나19 신규환자 밤 9시까지 7만497명   어제보다 4만5천여 명 많아', '전파력 강한 두 변이 함께 확산    정점 커지고 유행 길어질 듯 ', '코로나19 신규환자 밤 9시까지 7만497명   어제보다 4만5천여 명 많아', '50대 4차 접종 첫날 13만1천 명 예약   전체 50대 인구의 1 52%', '50대 4차 접종 시작    휴일 검사기관 확대 필요 ', '전파력 강한 두 변이 함께 확산    정점 커지고 유행 길어질 듯 ', ' 58세  이기일 복지부 2차관  코로나19 백신 4차접종', '월요일 기준 12주 만에 최다    더블링 3주 더 간다 ', '월요일 기준 12주 만에 최다    더블링 3주 더 간다 ', '정재훈 교수  코로나 재유행 급증 시기    더블링  3주 더 갈 것 ', ' 이슈인사이드  거리두기 없는데  노마스크     재유행 가속화 우려 ', '신규 환자 2만 6 299명   월요일 발표 기준 12주 만에 최다', '대통령실 참모진  코로나19 백신 4차 접종 동참', '신규 환자 2만 6 299명   월요일 발표 기준 12주 만에 최다', '중환자 병상 가동률 15 2%   재택환자 22만여 명', '신규 환자 26 299명   위중증 81명·사망 11명', ' 울산  울산  코로나19 재유행 대비 비상 방역대책 회의  열어', ' 현장영상+  당정 협의회  불공정 세제 공정하게 바로 잡을 것 ', ' 굿모닝브리핑  피서지로 확산되는 코로나  앞으로가 더 문제 ', '권성동  文 정권과 다른 과학방역 약속   일상 제약 최소화해야 ', '주말에도 4만 명대   오늘부터 50대 4차 접종', '오늘부터 지급기준 바뀐 격리자 생활지원비 온라인 신청', '50대·18세 이상 기저질환자 4차 백신 접종 오늘 시작', '대한항공  인천·삿포로 노선 2년여 만에 운항 재개', '내일부터 지급기준 바뀐 격리자 생활지원비 온라인 신청', '주말에도 4만 명대   내일부터 50대 4차 접종', '당정  30만 명 대비 병상 4천 개 추가 확보   과학 방역 ']

    word_dict = {}
    komoran = Komoran()
    stop_words = ['전체', '관련', '기준', '때문', '확인', '하루', '이틀']
    for title_text in titles:
        nouns = komoran.nouns(title_text)
        for noun in nouns:
            if len(noun) == 1:
                continue
            if noun not in stop_words:
                if noun not in word_counts:
                    word_dict[noun] = 0
                word_dict[noun] += 1
    print(word_dict)
    word_dict = {'접종': 8, '시작': 3, '휴일': 2, '검사': 2, '기관': 2, '확대': 2, '필요': 2, '피서지': 2, '코로나': 8, '확진': 1, '급증': 2, '지자체': 1,
     '방역': 4, '비상': 2, '19': 5, '신규': 5, '환자': 6, '어제': 2, '전파력': 2, '변이': 2, '확산': 3, '정점': 2, '유행': 5, '첫날': 1,
     '예약': 1, '인구': 1, '이기': 1, '복지부': 1, '차관': 1, '백신': 3, '월요일': 4, '최다': 4, '더블': 3, '간다': 2, '정재훈': 1, '교수': 1,
     '시기': 1, '이슈': 1, '사이드': 1, '거리': 1, '두기': 1, '마스크': 1, '가속': 1, '우려': 1, '발표': 2, '대통령': 1, '참모진': 1, '동참': 1,
     '중환자': 1, '병상': 2, '가동': 1, '재택': 1, '중증': 1, '사망': 1, '울산': 2, '대비': 2, '대책': 1, '회의': 1, '현장': 1, '영상': 1,
     '당정': 2, '협의회': 1, '불공': 1, '세제': 1, '공정': 1, '굿모닝': 1, '브리핑': 1, '앞으로': 1, '문제': 1, '권성동': 1, '정권': 1, '과학': 2,
     '약속': 1, '일상': 1, '제약': 1, '최소': 1, '주말': 2, '오늘': 3, '지급': 2, '격리': 2, '생활': 2, '지원비': 2, '온라인': 2, '신청': 2,
     '이상': 1, '기저': 1, '질환자': 1, '대한항공': 1, '인천': 1, '삿포로': 1, '노선': 1, '운항': 1, '재개': 1, '내일': 2, '추가': 1, '확보': 1}

    wc = WordCloud(
        font_path='c:/Windows/Fonts/malgun.ttf'
        , background_color='white'
        , width=800
        , height=800
        , mask=np.array(Image.open("./flask_plotly_SouthKorea2.jpg"))
    )

    # word_list = ' '.join(list(word_dict.keys()))
    # wc = wc.generate(word_list)
    wc_img = wc.generate_from_frequencies(word_dict)

    # fig = plt.figure(figsize=(8, 8))
    plt.title('코로나 확진자 추이 관련 뉴스 - 대표 키워드')
    # plt.imshow(wc_img)

    # -------------------------- 이미지 저장
    wc_img.to_file('./static/save_mying1111.jpg')
    # fig.savefig('./static/save_mying2222.jpg')
    return "save_mying1111.jpg"

SyntaxError: ignored